In [1]:
## for jupyter notebook only
import warnings

warnings.simplefilter("ignore")

from IPython.display import HTML, display

display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)


import json
import os

# Importing libraries
import pandas as pd
from ds.utilities.io import ds_trino

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.mode.chained_assignment = None


conn = ds_trino.create_trino_connection()

In [2]:
# VERY IMPORTAN: the query should include the catalag and the schema for every table e.g hudi_ingest.processed.fos_facility_view

query_por_sku = """

SELECT 
sm.uuid as sm_uuid,
sm_sku.line_item_uuid as line_item_id,
--facility.facility_uuid as facility_id,
org_uuid,
org.name as org_name,
sm.created_at,
cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ) as created_at_local,

created_by_user_uuid,
sm.accepted_at,
cast(AT_TIMEZONE(sm.accepted_at, facility.timezone) as TIMESTAMP ) as accepeted_at_local,

sm.delivered_at,
cast(AT_TIMEZONE(sm.delivered_at, facility.timezone) as TIMESTAMP ) as delivered_at_local,

delivery_window_start,
cast(AT_TIMEZONE(delivery_window_start, facility.timezone) as TIMESTAMP ) as delivery_windows_start_local,

delivery_window_end,
cast(AT_TIMEZONE(delivery_window_end, facility.timezone) as TIMESTAMP ) as delivery_windows_end_local,

sm.transaction_uuid,
sm.facility_uuid as facility_id,
facility.name as facility_name,
facility.country_code as country_code
,dc.country
,(CASE 

  when dc.country='Brazil' then 'Brazil'
  when dc.country = 'United States' then 'US/CAN'
  else 'SSL'
  end) as subregion

,(case when facility.country_code in ('MX','CO','BR') then 'LATAM'
        when facility.country_code in ('US') then 'US/CAN'
         end ) as region



,date_trunc('day',(DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP )))) as created_day
,date_trunc('month',DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))) as created_month
,date_trunc('week',DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))) as created_week_monday

,date_trunc('week', DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))+ interval '1' day)- interval '1' day as created_week


,sm_sku.sku_uuid
,sm_sku.quantity_expected
,sm_sku.quantity_delivered
,sku.name as sku_name
,sku.manufacturer_brand_name as brand_name
,sm.comment
, CASE 

  when sm.accepted_at IS NULL THEN 0
  Else 1
  END 

  as accepted
  
,CASE 

  when sm.deleted_at IS NULL THEN 0
  Else 1
  END 

  as deleted
  
,(
CASE
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when  cast(facility.facility_uuid as VARCHAR ) = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when  cast(facility.facility_uuid as VARCHAR ) = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when  cast(facility.facility_uuid as VARCHAR ) = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   
   else facility.name

  
END
) AS facility_sf_name

,(CASE
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        cast(facility.facility_uuid as VARCHAR ) IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
         cast(facility.facility_uuid as VARCHAR ) IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
        
        else 'other city'
END
) AS city




  
,space_uuid
,	expiration_date
,cast(AT_TIMEZONE(expiration_date, facility.timezone) as TIMESTAMP ) as expiration_date_local

,	live_at
,cast(AT_TIMEZONE(live_at, facility.timezone) as TIMESTAMP ) as live_at_local

,sm.delivery_sort
,cast(AT_TIMEZONE(sm.delivery_sort, facility.timezone) as TIMESTAMP ) as delivery_sort_local
,delivery_type, company_name, other_description

,shipment_receiving.vendor_expiration, facility_expiration, shipment_receiving.receiving_to_accepted_seconds, shipment_receiving.delivered_to_receiving_seconds
, status

FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest as sm

left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON sm.facility_uuid=cast(facility.facility_uuid as VARCHAR )

left join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest_line_item as sm_sku
on sm.uuid=cast(sm_sku.shipment_manifest_uuid as VARCHAR )

left JOIN hudi_ingest.skusku_restaurant_platform_cockroach_dbreplica.sku AS sku
on sm_sku.sku_uuid=cast(sku.sku_id as VARCHAR )

left join hudi_ingest.processed.store_config_organization_view as org
on sm.org_uuid=cast(org.organization_id as VARCHAR)

left join hudi_ingest.processed.inventory_shipment_receiving as shipment_receiving
on sm_sku.sku_uuid = shipment_receiving.sku_uuid
and cast(sm_sku.shipment_manifest_uuid as VARCHAR ) = shipment_receiving.shipment_manifest_uuid



JOIN hudi_ingest.scratch.dim_countries dc 
   ON facility.country_code = dc.country_code 
   --AND region = 'LATAM'


where facility.country_code in ('BR','MX','CO','US'--,'EC','PE','PR','CL','GT','CR','DO','SV'
)
and date_trunc('day',date(sm.day_partition)) >= DATE_ADD('Day' ,-90, current_date)
and sm.day_partition>='01-01-2021'
and sm_sku.day_partition>='01-01-2021'

and sku.day_partition>='01-01-2021'

"""

print("querying ...")
por_sku = ds_trino.fetch_data(conn=conn, sql_str=query_por_sku, use_cache=False)
print("pulled")

querying ...
pulled


In [3]:
import re
import json
import requests

In [4]:
from datetime import datetime, timedelta

In [5]:
import ast

In [6]:
por_sku['comment'].value_counts(dropna=False)

                                        200305
00016                                      397
1.0                                        350
00017                                      185
00021                                      169
                                         ...  
009e470d-7a94-44a7-9c10-83187076d595         1
c92c8eaa-3af1-494b-b531-dc1cc22fbcd0         1
5e2e1b2a-eb05-4e1d-bc38-5afc88e6b671         1
e83f9147-cbe6-442c-bc94-d4acbd37cbb9         1
8f35fecb-ac2a-4edb-bc60-a916ae6e0204         1
Name: comment, Length: 216, dtype: int64

In [7]:
types=[]
for i in por_sku['comment']:
    try:
        j=ast.literal_eval(i)
        #print(i,j)
        tipos=j.get("productTypes")
        #print(tipos, '\n')
        if len(tipos)==1:
            types.append(tipos[0])
        else:
            types.append(tipos)
    except:
        types.append('no comment')
        #print(tipos)
    

In [8]:
por_sku['types']=types
por_sku['types']

0         no comment
1         no comment
2         no comment
3         no comment
4         no comment
             ...    
208019    no comment
208020    no comment
208021    no comment
208022    no comment
208023    no comment
Name: types, Length: 208024, dtype: object

In [9]:
por_sku['types'].value_counts()

no comment    208024
Name: types, dtype: int64

In [10]:
fecha_dias=(datetime.today()-timedelta(days=45))

In [11]:
por_sku.dtypes

sm_uuid                            object
line_item_id                       object
org_uuid                           object
org_name                           object
created_at                         object
                                   ...   
facility_expiration                object
receiving_to_accepted_seconds     float64
delivered_to_receiving_seconds    float64
status                             object
types                              object
Length: 52, dtype: object

In [12]:
por_sku['created_at']=pd.to_datetime(por_sku['created_at'], errors='coerce')

In [13]:
por_sku['created_at_local']=pd.to_datetime(por_sku['created_at_local'], errors='coerce')

In [14]:
por_sku['accepted_at']=pd.to_datetime(por_sku['accepted_at'], errors='coerce')

In [15]:
por_sku['accepeted_at_local']=pd.to_datetime(por_sku['accepeted_at_local'], errors='coerce')

In [16]:
por_sku['delivered_at']=pd.to_datetime(por_sku['delivered_at'], errors='coerce')

In [17]:
por_sku['delivered_at_local']=pd.to_datetime(por_sku['delivered_at_local'], errors='coerce')

In [18]:
por_sku['delivery_window_start']=pd.to_datetime(por_sku['delivery_window_start'],errors='coerce')

In [19]:
por_sku['delivery_windows_start_local']=pd.to_datetime(por_sku['delivery_windows_start_local'],errors='coerce')

In [20]:
por_sku['delivery_window_end']=pd.to_datetime(por_sku['delivery_window_end'],errors='coerce')

In [21]:
por_sku['delivery_windows_end_local']=pd.to_datetime(por_sku['delivery_windows_end_local'],errors='coerce')

In [22]:
por_sku['expiration_date']=pd.to_datetime(por_sku['expiration_date'], errors='coerce')

In [23]:
por_sku['expiration_date_local']=pd.to_datetime(por_sku['expiration_date_local'], errors='coerce')

In [24]:
por_sku['live_at']=pd.to_datetime(por_sku['live_at'], errors='coerce')

In [25]:
por_sku['live_at_local']=pd.to_datetime(por_sku['live_at_local'], errors='coerce')

In [26]:
por_sku['delivery_sort']=pd.to_datetime(por_sku['delivery_sort'], errors='coerce')

In [27]:
por_sku['delivery_sort_local']=pd.to_datetime(por_sku['delivery_sort_local'], errors='coerce')

In [28]:
por_sku['anaquel_life_delta']=por_sku['expiration_date_local']-por_sku['delivered_at_local']

In [29]:
por_sku['anaquel_life_delta'].value_counts(dropna=False)

NaT                         70170
106 days 05:23:46.288000      127
104 days 07:08:10.414000      126
102 days 08:07:21             123
180 days 08:38:35.995000      100
                            ...  
-1 days +09:40:13               1
21 days 15:24:19.681000         1
252 days 03:12:59               1
344 days 04:59:20               1
9 days 07:06:58.026000          1
Name: anaquel_life_delta, Length: 19681, dtype: int64

In [30]:
por_sku['anaquel_life_days']=por_sku['anaquel_life_delta'].dt.components['days']+por_sku['anaquel_life_delta'].dt.components['hours']/24

In [31]:
por_sku['anaquel_life_days'].value_counts(dropna=False)

NaN           70170
9.166667       2086
6.166667       1747
9.208333       1736
6.208333       1596
              ...  
159.375000        1
457.166667        1
444.166667        1
357.458333        1
220.291667        1
Name: anaquel_life_days, Length: 3830, dtype: int64

In [32]:
query_codigo="""
SELECT * from hudi_ingest.skusku_restaurant_platform_cockroach_dbreplica.sku_barcode

where day_partition>='2020-01-01'
and deleted_at is null


"""

print("querying ...")
codigo = ds_trino.fetch_data(conn=conn, sql_str=query_codigo, use_cache=False)
print("pulled")

querying ...
pulled


In [33]:
codigo

,_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,barcode_type,barcode_value,sku_id,quantity,created_at,exist,_updated,create_time,deleted_at,_hoodie_is_deleted,day_partition
0,20230127080935157,20230127080935157_193_0,"barcode_type:BARCODE_CSS,barcode_value:8061445...",2021/12/09,5b099388-ef43-40c8-8d5a-8c980a63b078-0_193-26-...,BARCODE_CSS,806144560542,73273ed1-3717-4fcb-ad99-91451c9f1d0e,1,2021-12-09 23:37:36.385,None,1674802401796065651.0000000000,1639093056,None,False,2021-12-09
1,20230127065415416,20230127065415416_82_1,"barcode_type:BARCODE_UPC,barcode_value:9418701...",2021/12/09,5b099388-ef43-40c8-8d5a-8c980a63b078-0_193-26-...,BARCODE_UPC,94187015209,c185693c-dfbd-4641-9dbf-1963800e24d9,1,2021-12-09 01:58:12.536,None,1674802401796065651.0000000000,1639015092,None,False,2021-12-09
2,20230127065415416,20230127065415416_82_2,"barcode_type:BARCODE_UPC,barcode_value:6502586...",2021/12/09,5b099388-ef43-40c8-8d5a-8c980a63b078-0_193-26-...,BARCODE_UPC,650258648347,bcb1eb33-74a3-4262-9209-ab95c867717c,1,2021-12-09 14:02:18.304,None,1674802401796065651.0000000000,1639058538,None,False,2021-12-09
3,20230127080935157,20230127080935157_193_3,"barcode_type:BARCODE_CSS,barcode_value:8048961...",2021/12/09,5b099388-ef43-40c8-8d5a-8c980a63b078-0_193-26-...,BARCODE_CSS,804896129832,f60fcff0-40a9-4ae1-93ee-581f6ca0e2cd,1,2021-12-09 22:09:53.889,None,1674802401796065651.0000000000,1639087793,None,False,2021-12-09
4,20230127065415416,20230127065415416_82_4,"barcode_type:BARCODE_UPC,barcode_value:7689586...",2021/12/09,5b099388-ef43-40c8-8d5a-8c980a63b078-0_193-26-...,BARCODE_UPC,768958645786,5a4035c9-5164-4bad-9127-4ce7add29131,1,2021-12-09 17:10:39.211,None,1674802401796065651.0000000000,1639069839,None,False,2021-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102020,20230828214104787,20230828214104787_0_290,"barcode_type:BARCODE_CSS,barcode_value:5462021...",2023/08/28,d27920e2-857f-49ff-94d4-08db947c4372-0_0-26-18...,BARCODE_CSS,546202114554,4aa78e91-c622-403e-8f82-1e7d5c5fa806,1,2023-08-28 21:37:39.901,None,1693258659900668602.0000000000,1693258659,None,False,2023-08-28
102021,20230828214104787,20230828214104787_0_291,"barcode_type:BARCODE_CSS,barcode_value:7926111...",2023/08/28,d27920e2-857f-49ff-94d4-08db947c4372-0_0-26-18...,BARCODE_CSS,792611120352,d53262b0-296e-4988-92ad-da8250037f87,1,2023-08-28 21:37:39.901,None,1693258659900668602.0000000000,1693258659,None,False,2023-08-28
102022,20230828214104787,20230828214104787_0_292,"barcode_type:BARCODE_CSS,barcode_value:5720695...",2023/08/28,d27920e2-857f-49ff-94d4-08db947c4372-0_0-26-18...,BARCODE_CSS,572069569407,ae661230-00ac-48c1-8c79-e4c0cf7ab637,1,2023-08-28 21:37:39.901,None,1693258659900668602.0000000000,1693258659,None,False,2023-08-28
102023,20230828220729325,20230828220729325_0_293,"barcode_type:BARCODE_CSS,barcode_value:8873548...",2023/08/28,d27920e2-857f-49ff-94d4-08db947c4372-0_0-26-18...,BARCODE_CSS,887354853609,ba57f020-77e9-46ba-891f-212449314bf8,1,2023-08-28 22:03:47.502,None,1693260227502382837.0000000000,1693260227,None,False,2023-08-28


In [34]:
codigo_2=codigo.drop_duplicates().reset_index(drop=True)

In [35]:
dummies=pd.get_dummies(data=codigo_2['barcode_type'], columns=['barcode_type'])


In [36]:
dummies_2=dummies.multiply(codigo_2['barcode_value'], axis=0)

In [37]:
codigo_3=pd.concat([codigo_2,dummies_2],axis=1)

In [38]:
codigo_3.columns

Index(['_hoodie_commit_time', '_hoodie_commit_seqno', '_hoodie_record_key',
       '_hoodie_partition_path', '_hoodie_file_name', 'barcode_type',
       'barcode_value', 'sku_id', 'quantity', 'created_at', 'exist',
       '_updated', 'create_time', 'deleted_at', '_hoodie_is_deleted',
       'day_partition', 'BARCODE_ASIN', 'BARCODE_CSS', 'BARCODE_GTIN',
       'BARCODE_UPC'],
      dtype='object')

In [39]:
codigo_4=pd.pivot_table(data=codigo_3, index=['sku_id',], values=['BARCODE_CSS',
       'BARCODE_GTIN', 'BARCODE_UPC'],  aggfunc=lambda x:x).reset_index()

In [40]:
codigo_4

,sku_id,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,000007d6-2e91-4b1d-8d41-672ac2fada4a,982399961654,,
1,0000c96e-c37c-4bfd-ba93-540a0198ad3a,269359827673,,
2,00011928-360e-43a7-94df-d91ac6e78539,,,7898953903560
3,00012843-1d6b-4d1c-88b3-a4e36eba3166,982939255409,,
4,0002a884-8e32-4f8d-bb83-ef21f7c23287,,,7707159300199
...,...,...,...,...
84582,fffd7e3a-b9c4-4bee-9557-699e571e5cfc,458700348997,,
84583,fffdfa4d-ab33-47f0-a30f-e77ff9012299,471260095549,,
84584,fffe7698-ad9f-486d-92bb-e10deeb4980e,515174340020,,
84585,ffff14db-5b90-4859-b5d1-5dbaa8ec9609,,,ザ・ガーデン自由が丘／スグル やみつき炙り鱈


In [41]:
def limpio(myList):
    try:
        first=' '.join(myList)
        second=first.strip()
        tercero=second.replace(' ','')
        return tercero
    except:
        return myList


In [42]:
codigo_4['BARCODE_CSS']=codigo_4['BARCODE_CSS'].apply(limpio)

In [43]:
codigo_4['BARCODE_GTIN']=codigo_4['BARCODE_GTIN'].apply(limpio)

In [44]:
codigo_4['BARCODE_UPC']=codigo_4['BARCODE_UPC'].apply(limpio)

In [45]:
codigo_4.columns

Index(['sku_id', 'BARCODE_CSS', 'BARCODE_GTIN', 'BARCODE_UPC'], dtype='object')

In [46]:
codigo_4

,sku_id,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,000007d6-2e91-4b1d-8d41-672ac2fada4a,982399961654,,
1,0000c96e-c37c-4bfd-ba93-540a0198ad3a,269359827673,,
2,00011928-360e-43a7-94df-d91ac6e78539,,,7898953903560
3,00012843-1d6b-4d1c-88b3-a4e36eba3166,982939255409,,
4,0002a884-8e32-4f8d-bb83-ef21f7c23287,,,7707159300199
...,...,...,...,...
84582,fffd7e3a-b9c4-4bee-9557-699e571e5cfc,458700348997,,
84583,fffdfa4d-ab33-47f0-a30f-e77ff9012299,471260095549,,
84584,fffe7698-ad9f-486d-92bb-e10deeb4980e,515174340020,,
84585,ffff14db-5b90-4859-b5d1-5dbaa8ec9609,,,ザ・ガーデン自由が丘／スグル やみつき炙り鱈


In [47]:
codigo_merge=codigo_4[[ 'sku_id','BARCODE_CSS', 'BARCODE_GTIN', 'BARCODE_UPC']].drop_duplicates(subset=[ 'sku_id']).reset_index(drop=True)

In [48]:
codigo_merge

,sku_id,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,000007d6-2e91-4b1d-8d41-672ac2fada4a,982399961654,,
1,0000c96e-c37c-4bfd-ba93-540a0198ad3a,269359827673,,
2,00011928-360e-43a7-94df-d91ac6e78539,,,7898953903560
3,00012843-1d6b-4d1c-88b3-a4e36eba3166,982939255409,,
4,0002a884-8e32-4f8d-bb83-ef21f7c23287,,,7707159300199
...,...,...,...,...
84582,fffd7e3a-b9c4-4bee-9557-699e571e5cfc,458700348997,,
84583,fffdfa4d-ab33-47f0-a30f-e77ff9012299,471260095549,,
84584,fffe7698-ad9f-486d-92bb-e10deeb4980e,515174340020,,
84585,ffff14db-5b90-4859-b5d1-5dbaa8ec9609,,,ザ・ガーデン自由が丘／スグル やみつき炙り鱈


In [49]:
len(por_sku)

208024

In [50]:
#por_sku.columns#

In [51]:
por_sku_2=por_sku.merge(codigo_merge,how='left',left_on=['sku_uuid'],
              right_on=['sku_id'],suffixes=('', '_bar_code'))

In [52]:
por_sku_2.columns

Index(['sm_uuid', 'line_item_id', 'org_uuid', 'org_name', 'created_at',
       'created_at_local', 'created_by_user_uuid', 'accepted_at',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'delivery_window_start', 'delivery_windows_start_local',
       'delivery_window_end', 'delivery_windows_end_local', 'transaction_uuid',
       'facility_id', 'facility_name', 'country_code', 'country', 'subregion',
       'region', 'created_day', 'created_month', 'created_week_monday',
       'created_week', 'sku_uuid', 'quantity_expected', 'quantity_delivered',
       'sku_name', 'brand_name', 'comment', 'accepted', 'deleted',
       'facility_sf_name', 'city', 'space_uuid', 'expiration_date',
       'expiration_date_local', 'live_at', 'live_at_local', 'delivery_sort',
       'delivery_sort_local', 'delivery_type', 'company_name',
       'other_description', 'vendor_expiration', 'facility_expiration',
       'receiving_to_accepted_seconds', 'delivered_to_receiving_seconds',
 

In [53]:
por_sku_2

,sm_uuid,line_item_id,org_uuid,org_name,created_at,created_at_local,created_by_user_uuid,accepted_at,accepeted_at_local,delivered_at,delivered_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_id,facility_name,country_code,country,subregion,region,created_day,created_month,created_week_monday,created_week,sku_uuid,quantity_expected,quantity_delivered,sku_name,brand_name,comment,accepted,deleted,facility_sf_name,city,space_uuid,expiration_date,expiration_date_local,live_at,live_at_local,delivery_sort,delivery_sort_local,delivery_type,company_name,other_description,vendor_expiration,facility_expiration,receiving_to_accepted_seconds,delivered_to_receiving_seconds,status,types,anaquel_life_delta,anaquel_life_days,sku_id,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,2172bee4-5291-45f6-af55-2a146b9f6179,9377d096-d2c7-4370-bd72-e24360cb6fd5,6da4b452-29c0-492c-b3b2-72d8aabd6bf1,Tecas Pâtisserie - Pik n' Pak,2023-08-15 18:03:50.939,2023-08-15 15:03:50.939,31db63dc-dd65-4acb-bb26-b00e3f2693d8,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,2023-08-15 19:00:00,2023-08-15 16:00:00,2023-08-15 20:00:00,2023-08-15 17:00:00,2f56c39d-991e-4018-af5c-50ba01c42d60,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,BR,Brazil,Brazil,LATAM,2023-08-15,2023-08-01,2023-08-14,2023-08-13,13e151a9-46db-4765-be19-fddae8b4642a,0,3.0,Bolo no Pote - Prestígio,Tecas Pâtisserie,,1,0,SAO - Dino,Sao Paulo,335ff825-71eb-4c50-bd5a-2d6979811808,2023-09-03,2023-09-02 21:00:00,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,COURIER,lalamove,None,2023-09-03,[2023-09-03],2210.0,0.0,ACCEPTED,no comment,18 days 02:50:54.696000,18.083333,13e151a9-46db-4765-be19-fddae8b4642a,632454449269,8949461894905,
1,2172bee4-5291-45f6-af55-2a146b9f6179,93e3d3c2-0f72-45e7-a6a1-e93912c0bdbf,6da4b452-29c0-492c-b3b2-72d8aabd6bf1,Tecas Pâtisserie - Pik n' Pak,2023-08-15 18:03:50.939,2023-08-15 15:03:50.939,31db63dc-dd65-4acb-bb26-b00e3f2693d8,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,2023-08-15 19:00:00,2023-08-15 16:00:00,2023-08-15 20:00:00,2023-08-15 17:00:00,2f56c39d-991e-4018-af5c-50ba01c42d60,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,BR,Brazil,Brazil,LATAM,2023-08-15,2023-08-01,2023-08-14,2023-08-13,13e151a9-46db-4765-be19-fddae8b4642a,3,0.0,Bolo no Pote - Prestígio,Tecas Pâtisserie,,1,0,SAO - Dino,Sao Paulo,None,2023-09-03,2023-09-02 21:00:00,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,COURIER,lalamove,None,2023-09-03,[2023-09-03],2210.0,0.0,ACCEPTED,no comment,18 days 02:50:54.696000,18.083333,13e151a9-46db-4765-be19-fddae8b4642a,632454449269,8949461894905,
2,98ec5cc9-27e7-439e-b5fa-d24836b20f65,bc5bfcfd-13c8-40e9-9c44-970679e5c448,2b58d349-dd0e-474e-b4bb-51d705aaea9a,Éclair Moi - Pik n' Pak,2023-07-21 12:41:49.745,2023-07-21 09:41:49.745,622327ed-57bd-400f-8b66-5a24a25a6a56,2023-07-21 23:06:40.440,2023-07-21 20:06:40.440,2023-07-21 22:23:40.000,2023-07-21 19:23:40.000,2023-07-21 18:30:00,2023-07-21 15:30:00,2023-07-21 19:00:00,2023-07-21 16:00:00,f1199b0e-5c16-46f5-a71e-fca238f85ebe,4326199f-628a-4b00-a6e6-054fb5182c64,BR - Sao Paulo - Guararapes,BR,Brazil,Brazil,LATAM,2023-07-21,2023-07-01,2023-07-17,2023-07-16,31c0c39c-4cc7-483b-a46e-0c71497b5988,0,5.0,Eclair Chocolate Amargo,Éclair Moi - Pik n' Pak,,1,0,SAO - Guararapes (Brooklin),Sao Paulo,99944d23-eb52-4450-a1c6-daddd7df575c,2023-11-21,2023-11-20 21:00:00,2023-07-21 23:06:40.440,2023-07-21 20:06:40.440,2023-07-21 22:23:40.000,2023-07-21 19:23:40.000,OTHER,None,Sera entregue via motoboy Uber,2023-11-21,[2023-11-21],99.0,2481.0,ACCEPTED,no comment,122 days 01:36:20,122.041667,31c0c39c-4cc7-483b-a46e-0c71497b5988,757314104485,,
3,98ec5cc9-27e7-439e-b5fa-d24836b20f65,a9d838f0-c973-40f0-8598-824923d9c881,2b58d349-dd0e-474e-b4bb-51d705

In [54]:
por_sku_2.isnull().sum()

sm_uuid                  0
line_item_id             0
org_uuid                 0
org_name                 0
created_at               0
                     ...  
anaquel_life_days    70170
sku_id                  61
BARCODE_CSS             61
BARCODE_GTIN            61
BARCODE_UPC             61
Length: 58, dtype: int64

In [55]:
por_sku_3=por_sku_2[[
'sm_uuid', 'line_item_id', 'org_uuid', 'org_name', 'created_at',
       'created_at_local', 'created_by_user_uuid', 'accepted_at',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'delivery_window_start', 'delivery_windows_start_local',
       'delivery_window_end', 'delivery_windows_end_local', 'transaction_uuid',
       'facility_id', 'facility_name', 'country_code', 'country', 'subregion',
       'region', 'created_day', 'created_month', 'created_week_monday',
       'created_week', 'sku_uuid', 'quantity_expected', 'quantity_delivered',
       'sku_name', 'brand_name', 'comment', 'accepted', 'deleted',
       'facility_sf_name', 'city', 'space_uuid', 'expiration_date',
       'expiration_date_local', 'live_at', 'live_at_local', 'delivery_sort',
       'delivery_sort_local', 'delivery_type', 'company_name',
       'other_description', 'vendor_expiration', 'facility_expiration',
       'receiving_to_accepted_seconds', 'delivered_to_receiving_seconds',
       'status', 'types', 'anaquel_life_delta', 'anaquel_life_days',  
 'BARCODE_CSS','BARCODE_GTIN', 'BARCODE_UPC'
    
]]

In [56]:
por_sku_spread=por_sku_3[por_sku_3['created_at_local']>=fecha_dias].reset_index(drop=True)

In [57]:
por_sku_spread

,sm_uuid,line_item_id,org_uuid,org_name,created_at,created_at_local,created_by_user_uuid,accepted_at,accepeted_at_local,delivered_at,delivered_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_id,facility_name,country_code,country,subregion,region,created_day,created_month,created_week_monday,created_week,sku_uuid,quantity_expected,quantity_delivered,sku_name,brand_name,comment,accepted,deleted,facility_sf_name,city,space_uuid,expiration_date,expiration_date_local,live_at,live_at_local,delivery_sort,delivery_sort_local,delivery_type,company_name,other_description,vendor_expiration,facility_expiration,receiving_to_accepted_seconds,delivered_to_receiving_seconds,status,types,anaquel_life_delta,anaquel_life_days,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,2172bee4-5291-45f6-af55-2a146b9f6179,9377d096-d2c7-4370-bd72-e24360cb6fd5,6da4b452-29c0-492c-b3b2-72d8aabd6bf1,Tecas Pâtisserie - Pik n' Pak,2023-08-15 18:03:50.939,2023-08-15 15:03:50.939,31db63dc-dd65-4acb-bb26-b00e3f2693d8,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,2023-08-15 19:00:00,2023-08-15 16:00:00,2023-08-15 20:00:00,2023-08-15 17:00:00,2f56c39d-991e-4018-af5c-50ba01c42d60,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,BR,Brazil,Brazil,LATAM,2023-08-15,2023-08-01,2023-08-14,2023-08-13,13e151a9-46db-4765-be19-fddae8b4642a,0,3.0,Bolo no Pote - Prestígio,Tecas Pâtisserie,,1,0,SAO - Dino,Sao Paulo,335ff825-71eb-4c50-bd5a-2d6979811808,2023-09-03,2023-09-02 21:00:00,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,COURIER,lalamove,None,2023-09-03,[2023-09-03],2210.0,0.0,ACCEPTED,no comment,18 days 02:50:54.696000,18.083333,632454449269,8949461894905,
1,2172bee4-5291-45f6-af55-2a146b9f6179,93e3d3c2-0f72-45e7-a6a1-e93912c0bdbf,6da4b452-29c0-492c-b3b2-72d8aabd6bf1,Tecas Pâtisserie - Pik n' Pak,2023-08-15 18:03:50.939,2023-08-15 15:03:50.939,31db63dc-dd65-4acb-bb26-b00e3f2693d8,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,2023-08-15 19:00:00,2023-08-15 16:00:00,2023-08-15 20:00:00,2023-08-15 17:00:00,2f56c39d-991e-4018-af5c-50ba01c42d60,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,BR,Brazil,Brazil,LATAM,2023-08-15,2023-08-01,2023-08-14,2023-08-13,13e151a9-46db-4765-be19-fddae8b4642a,3,0.0,Bolo no Pote - Prestígio,Tecas Pâtisserie,,1,0,SAO - Dino,Sao Paulo,None,2023-09-03,2023-09-02 21:00:00,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,COURIER,lalamove,None,2023-09-03,[2023-09-03],2210.0,0.0,ACCEPTED,no comment,18 days 02:50:54.696000,18.083333,632454449269,8949461894905,
2,f6286ac0-87e0-4ca4-9ba6-a05642136223,f6b03d4a-1314-48cd-87e8-ac8334d98993,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,Franui,2023-08-16 00:19:19.641,2023-08-15 19:19:19.641,6cbb0282-3d21-47df-83e7-693b7b52d62e,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,2023-08-16 14:30:00,2023-08-16 09:30:00,2023-08-16 17:45:00,2023-08-16 12:45:00,84e433bf-efbb-4a7b-8547-3c5e80844f49,4d78d1e1-5bce-4307-8175-51c816120fce,SA-CO-Bogotá-Castellana,CO,Colombia,SSL,LATAM,2023-08-15,2023-08-01,2023-08-14,2023-08-13,be644db4-428a-4b32-b32f-e13cea01227a,0,48.0,Franuí chocolate con leche,Franui,,1,0,BOG - Castellana,Bogota,2e6fad0f-0593-4e06-9658-175fd528b008,NaT,NaT,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,COURIER,TODOS COMEMOS,None,None,[None],41.0,0.0,ACCEPTED,no comment,NaT,NaN,288768753447,7798147783223,
3,f6286ac0-87e0-4ca4-9ba6-a05642136223,466aaff1-7bcd-4125-acd8-adfcb7e6b274,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,Franui,2023-08-16 00:19:19.641,2023-08-15 19:19:19.641,6cbb0282-3d21-47df-83e7-693b7b52d62e,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,2023-08-16 14:30:00,2023-08-16 09

In [58]:
por_sku_spread.columns

Index(['sm_uuid', 'line_item_id', 'org_uuid', 'org_name', 'created_at',
       'created_at_local', 'created_by_user_uuid', 'accepted_at',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'delivery_window_start', 'delivery_windows_start_local',
       'delivery_window_end', 'delivery_windows_end_local', 'transaction_uuid',
       'facility_id', 'facility_name', 'country_code', 'country', 'subregion',
       'region', 'created_day', 'created_month', 'created_week_monday',
       'created_week', 'sku_uuid', 'quantity_expected', 'quantity_delivered',
       'sku_name', 'brand_name', 'comment', 'accepted', 'deleted',
       'facility_sf_name', 'city', 'space_uuid', 'expiration_date',
       'expiration_date_local', 'live_at', 'live_at_local', 'delivery_sort',
       'delivery_sort_local', 'delivery_type', 'company_name',
       'other_description', 'vendor_expiration', 'facility_expiration',
       'receiving_to_accepted_seconds', 'delivered_to_receiving_seconds',
 

In [59]:
por_sku_spread_2=por_sku_spread[['sm_uuid', 'org_uuid', 'org_name',  'created_at_local',
                                 'created_by_user_uuid',
        'accepeted_at_local',
       'delivered_at_local',
       'delivery_windows_start_local', 
       'delivery_windows_end_local', 'transaction_uuid', 'facility_name',
       'country_code', 'country', 'subregion', 'region','city',
         'created_day', 'created_month','created_week', 'sku_uuid', 
        'quantity_expected', 'quantity_delivered',
       'sku_name', 'brand_name', 'comment', 'accepted', 'deleted',
       'facility_sf_name', 'space_uuid', 'expiration_date',
       'expiration_date_local', 'live_at', 'live_at_local', 'delivery_sort',
       'delivery_sort_local', 'delivery_type', 'anaquel_life_delta',
       'anaquel_life_days','company_name', 'vendor_expiration', 'facility_expiration',
       'other_description', 'status', 'BARCODE_CSS',
       'BARCODE_GTIN', 'BARCODE_UPC']]

In [60]:
#por_sku_spread_2['types'].value_counts()

In [61]:
por_sku_spread_2

,sm_uuid,org_uuid,org_name,created_at_local,created_by_user_uuid,accepeted_at_local,delivered_at_local,delivery_windows_start_local,delivery_windows_end_local,transaction_uuid,facility_name,country_code,country,subregion,region,city,created_day,created_month,created_week,sku_uuid,quantity_expected,quantity_delivered,sku_name,brand_name,comment,accepted,deleted,facility_sf_name,space_uuid,expiration_date,expiration_date_local,live_at,live_at_local,delivery_sort,delivery_sort_local,delivery_type,anaquel_life_delta,anaquel_life_days,company_name,vendor_expiration,facility_expiration,other_description,status,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,2172bee4-5291-45f6-af55-2a146b9f6179,6da4b452-29c0-492c-b3b2-72d8aabd6bf1,Tecas Pâtisserie - Pik n' Pak,2023-08-15 15:03:50.939,31db63dc-dd65-4acb-bb26-b00e3f2693d8,2023-08-15 18:45:55.449,2023-08-15 18:09:05.304,2023-08-15 16:00:00,2023-08-15 17:00:00,2f56c39d-991e-4018-af5c-50ba01c42d60,BR - Sao Paulo - Dino,BR,Brazil,Brazil,LATAM,Sao Paulo,2023-08-15,2023-08-01,2023-08-13,13e151a9-46db-4765-be19-fddae8b4642a,0,3.0,Bolo no Pote - Prestígio,Tecas Pâtisserie,,1,0,SAO - Dino,335ff825-71eb-4c50-bd5a-2d6979811808,2023-09-03,2023-09-02 21:00:00,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,COURIER,18 days 02:50:54.696000,18.083333,lalamove,2023-09-03,[2023-09-03],None,ACCEPTED,632454449269,8949461894905,
1,2172bee4-5291-45f6-af55-2a146b9f6179,6da4b452-29c0-492c-b3b2-72d8aabd6bf1,Tecas Pâtisserie - Pik n' Pak,2023-08-15 15:03:50.939,31db63dc-dd65-4acb-bb26-b00e3f2693d8,2023-08-15 18:45:55.449,2023-08-15 18:09:05.304,2023-08-15 16:00:00,2023-08-15 17:00:00,2f56c39d-991e-4018-af5c-50ba01c42d60,BR - Sao Paulo - Dino,BR,Brazil,Brazil,LATAM,Sao Paulo,2023-08-15,2023-08-01,2023-08-13,13e151a9-46db-4765-be19-fddae8b4642a,3,0.0,Bolo no Pote - Prestígio,Tecas Pâtisserie,,1,0,SAO - Dino,None,2023-09-03,2023-09-02 21:00:00,2023-08-15 21:45:55.449,2023-08-15 18:45:55.449,2023-08-15 21:09:05.304,2023-08-15 18:09:05.304,COURIER,18 days 02:50:54.696000,18.083333,lalamove,2023-09-03,[2023-09-03],None,ACCEPTED,632454449269,8949461894905,
2,f6286ac0-87e0-4ca4-9ba6-a05642136223,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,Franui,2023-08-15 19:19:19.641,6cbb0282-3d21-47df-83e7-693b7b52d62e,2023-08-16 09:59:44.895,2023-08-16 09:59:03.765,2023-08-16 09:30:00,2023-08-16 12:45:00,84e433bf-efbb-4a7b-8547-3c5e80844f49,SA-CO-Bogotá-Castellana,CO,Colombia,SSL,LATAM,Bogota,2023-08-15,2023-08-01,2023-08-13,be644db4-428a-4b32-b32f-e13cea01227a,0,48.0,Franuí chocolate con leche,Franui,,1,0,BOG - Castellana,2e6fad0f-0593-4e06-9658-175fd528b008,NaT,NaT,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,COURIER,NaT,NaN,TODOS COMEMOS,None,[None],None,ACCEPTED,288768753447,7798147783223,
3,f6286ac0-87e0-4ca4-9ba6-a05642136223,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,Franui,2023-08-15 19:19:19.641,6cbb0282-3d21-47df-83e7-693b7b52d62e,2023-08-16 09:59:44.895,2023-08-16 09:59:03.765,2023-08-16 09:30:00,2023-08-16 12:45:00,84e433bf-efbb-4a7b-8547-3c5e80844f49,SA-CO-Bogotá-Castellana,CO,Colombia,SSL,LATAM,Bogota,2023-08-15,2023-08-01,2023-08-13,be644db4-428a-4b32-b32f-e13cea01227a,48,0.0,Franuí chocolate con leche,Franui,,1,0,BOG - Castellana,None,NaT,NaT,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,COURIER,NaT,NaN,TODOS COMEMOS,None,[None],None,ACCEPTED,288768753447,7798147783223,
4,bd13fe31-d10d-45a7-aa7e-41003f0a1559,a39980f7-56a2-48de-9b4a-e23acb05b665,Delu's Confeitaria,2023-08-29 21:54:21.505,c478bebe-14e7-4dab-b30b-fddb159ba945,2023-08-30 11:08:29.326,2023-08-30 10:50:21.909,2023-08-31 09:30:00,2023-08-31 11:00:00,741592ca-ad55-42b4-9d3a-a9bdd6b83f0d,SA-BR-BH-Rua Paracatu,BR,Brazil,Brazil,LATAM,Belo Horizonte,2023-08-29,2023-08-01,2023-08-27,b30ba3f8-4691-453b-b82d-3585aa8b25f8,0,5.0,Bolo de Cenoura - Délu's,Délu's Confeitaria,,1,0,BHZ - Paracatu,7f7a5559-469a-41dd-a7be-f5e333b543c1,2023-09-07,20

In [62]:
por_sku_spread_ssl=por_sku_spread_2[por_sku_spread_2['country'].isin(['Mexico','Colombia'])].reset_index(drop=True)

In [63]:
por_sku_spread_ssl

,sm_uuid,org_uuid,org_name,created_at_local,created_by_user_uuid,accepeted_at_local,delivered_at_local,delivery_windows_start_local,delivery_windows_end_local,transaction_uuid,facility_name,country_code,country,subregion,region,city,created_day,created_month,created_week,sku_uuid,quantity_expected,quantity_delivered,sku_name,brand_name,comment,accepted,deleted,facility_sf_name,space_uuid,expiration_date,expiration_date_local,live_at,live_at_local,delivery_sort,delivery_sort_local,delivery_type,anaquel_life_delta,anaquel_life_days,company_name,vendor_expiration,facility_expiration,other_description,status,BARCODE_CSS,BARCODE_GTIN,BARCODE_UPC
0,f6286ac0-87e0-4ca4-9ba6-a05642136223,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,Franui,2023-08-15 19:19:19.641,6cbb0282-3d21-47df-83e7-693b7b52d62e,2023-08-16 09:59:44.895,2023-08-16 09:59:03.765,2023-08-16 09:30:00,2023-08-16 12:45:00,84e433bf-efbb-4a7b-8547-3c5e80844f49,SA-CO-Bogotá-Castellana,CO,Colombia,SSL,LATAM,Bogota,2023-08-15,2023-08-01,2023-08-13,be644db4-428a-4b32-b32f-e13cea01227a,0,48.0,Franuí chocolate con leche,Franui,,1,0,BOG - Castellana,2e6fad0f-0593-4e06-9658-175fd528b008,NaT,NaT,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,COURIER,NaT,NaN,TODOS COMEMOS,None,[None],None,ACCEPTED,288768753447,7798147783223,
1,f6286ac0-87e0-4ca4-9ba6-a05642136223,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,Franui,2023-08-15 19:19:19.641,6cbb0282-3d21-47df-83e7-693b7b52d62e,2023-08-16 09:59:44.895,2023-08-16 09:59:03.765,2023-08-16 09:30:00,2023-08-16 12:45:00,84e433bf-efbb-4a7b-8547-3c5e80844f49,SA-CO-Bogotá-Castellana,CO,Colombia,SSL,LATAM,Bogota,2023-08-15,2023-08-01,2023-08-13,be644db4-428a-4b32-b32f-e13cea01227a,48,0.0,Franuí chocolate con leche,Franui,,1,0,BOG - Castellana,None,NaT,NaT,2023-08-16 14:59:44.895,2023-08-16 09:59:44.895,2023-08-16 14:59:03.765,2023-08-16 09:59:03.765,COURIER,NaT,NaN,TODOS COMEMOS,None,[None],None,ACCEPTED,288768753447,7798147783223,
2,fcdf193c-48ce-462b-8cd7-09a254491d28,50101211-a3dc-4621-945a-66eadc6de59b,Casa Pave,2023-09-06 09:30:09.935,6f106da3-30cb-4210-b0a6-2f61512612ee,2023-09-06 12:09:30.365,2023-09-06 11:47:47.032,2023-09-06 11:30:00,2023-09-06 13:00:00,0e62be0a-ccca-405e-a092-841d7480dc37,SA-CO-Bogotá-Atabanza,CO,Colombia,SSL,LATAM,Bogota,2023-09-06,2023-09-01,2023-09-03,d3f53f24-4248-4a03-964f-ae5e778dabb3,0,2.0,Porción Pavé Milo,Casa Pave,,1,0,BOG - Atabanza,a0d2e894-0d6b-4b5e-8713-9994d635c47f,2023-09-08,2023-09-07 19:00:00,2023-09-06 17:09:30.365,2023-09-06 12:09:30.365,2023-09-06 16:47:47.032,2023-09-06 11:47:47.032,COURIER,1 days 07:12:12.968000,1.291667,,2023-09-08,[2023-09-08],None,ACCEPTED,884836342476,,
3,fcdf193c-48ce-462b-8cd7-09a254491d28,50101211-a3dc-4621-945a-66eadc6de59b,Casa Pave,2023-09-06 09:30:09.935,6f106da3-30cb-4210-b0a6-2f61512612ee,2023-09-06 12:09:30.365,2023-09-06 11:47:47.032,2023-09-06 11:30:00,2023-09-06 13:00:00,0e62be0a-ccca-405e-a092-841d7480dc37,SA-CO-Bogotá-Atabanza,CO,Colombia,SSL,LATAM,Bogota,2023-09-06,2023-09-01,2023-09-03,1829d27e-28dc-455d-a59d-7033c0f96cb9,0,2.0,Porción Pavé Amor y Amistad,Casa Pave,,1,0,BOG - Atabanza,a0d2e894-0d6b-4b5e-8713-9994d635c47f,2023-09-14,2023-09-13 19:00:00,2023-09-06 17:09:30.365,2023-09-06 12:09:30.365,2023-09-06 16:47:47.032,2023-09-06 11:47:47.032,COURIER,7 days 07:12:12.968000,7.291667,,2023-09-14,[2023-09-14],None,ACCEPTED,856285460869,,856285460869
4,fcdf193c-48ce-462b-8cd7-09a254491d28,50101211-a3dc-4621-945a-66eadc6de59b,Casa Pave,2023-09-06 09:30:09.935,6f106da3-30cb-4210-b0a6-2f61512612ee,2023-09-06 12:09:30.365,2023-09-06 11:47:47.032,2023-09-06 11:30:00,2023-09-06 13:00:00,0e62be0a-ccca-405e-a092-841d7480dc37,SA-CO-Bogotá-Atabanza,CO,Colombia,SSL,LATAM,Bogota,2023-09-06,2023-09-01,2023-09-03,1829d27e-28dc-455d-a59d-7033c0f96cb9,2,0.0,Porción Pavé Amor y Amistad,Casa Pave,,1,0,BOG - Atabanza,None,2023-09-14,2023-09-13 19:00:00,2023-09-06 17:09:30.365,2023-09-06 12:09:30.365,2023-09-06 16:47:47.032,2023-09-06 11:47:47.032,CO

In [64]:
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc = 

In [65]:
import pygsheets
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1Ffh0lxSI31wEckOrbpB1gxgd4ry3GdXO62XvksepGds')

# select sheet

ws = ss.worksheet('title','data_por_sku')

print('Clearing Range ...')
ws.clear(start='A', end='AQ')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(por_sku_spread_ssl, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [66]:
#import pygsheets
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
#gc = pygsheets.authorize(custom_credentials=my_credentials)


# open the spreedsheet
print('Opening Spreadsheet')

#ss = gc.open_by_key('1BG9l8Rzo-BTKdIPh8D4WaOPa-KWw5hFAecC9mpmjfYw')

# select sheet

#ws = ss.worksheet('title','Data Set sku level')

print('Clearing Range ...')
#ws.clear(start='A', end='AQ')
print('Range CLEARED')

print('Pasting Info ...')
#ws.set_dataframe(por_sku_spread_2, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [67]:
query_por_sm = """


with shipment_line_item as (
SELECT
shipment_manifest_uuid,
count(quantity_expected) as num_skus,
sum(case 
when quantity_delivered is null then 1
else 0
end) as non_counted_sku,
sum(case
when quantity_delivered is not null then 1
else 0
end) as counted_sku,

sum(case
when expiration_date is not null  then 1
else 0
end) as has_expiration_date,

sum(case
when space_uuid is not null then 1
else 0
end) as has_space_uuid,

sum(case
when live_at is not null then 1
else 0
end) as has_live_at,





sum(li.quantity_expected) as total_quantity_expected,
sum(li.quantity_delivered) as total_quantity_delivered
FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest_line_item as li

where li.day_partition >='2020-01-01'
group by 1
)


SELECT 
--uuid,
shipment_line_item.*,
facility.facility_uuid as facility_uuid,
org_uuid,
--created_by_user_uuid,
sm.accepted_at,
sm.created_at,
cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ) as created_at_local,
cast(AT_TIMEZONE(sm.accepted_at, facility.timezone) as TIMESTAMP ) as accepeted_at_local,
delivered_at,
cast(AT_TIMEZONE(sm.delivered_at, facility.timezone) as TIMESTAMP ) as delivered_at_local,

sm.deleted_at as deleted_sm_date,
cast(AT_TIMEZONE(sm.deleted_at, facility.timezone) as TIMESTAMP ) as deleted_at_local,

delivery_window_start,
cast(AT_TIMEZONE(delivery_window_start, facility.timezone) as TIMESTAMP ) as delivery_windows_start_local,

--delivery_window_start,
--date(cast(AT_TIMEZONE(delivery_window_start, facility.timezone) as TIMESTAMP ) ) as delivery_windows_start_date,

delivery_window_end,
cast(AT_TIMEZONE(delivery_window_end, facility.timezone) as TIMESTAMP ) as delivery_windows_end_local,

transaction_uuid,
facility.name as facility_name,
facility.timezone,
org.name as org_name
,facility.country_code
,dc.country
,date_trunc('day',date(sm.created_at)) as created_day
,date_trunc('month',date(sm.created_at)) as created_month

,date_trunc('week', DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))+ interval '1' day)- interval '1' day as created_week


--,date_trunc('week',date(sm.created_at)) as created_week
, sm.comment as comment
, CASE 

  when accepted_at IS NULL THEN 0
  Else 1
  END 

  as accepted_num
  
,CASE 

  when sm.deleted_at IS NULL THEN 0
  Else 1
  END 

  as deleted_num
  
  
,CASE 

  when sm.created_at IS NULL THEN 0
  Else 1
  END 

  as created_num  
  
,CASE 

  when sm.receiving_started IS NULL THEN 0
  Else 1
  END 

  as receiving_started_num  
  
,CASE 

  when sm.delivered_at IS NULL THEN 0
  Else 1
  END 

  as delivered_num 
  

,contains_dry_product_type,
contains_cold_product_type,
contains_frozen_product_type,
delivery_type,
company_name,
tracking_id,
deliverer_name,
deliverer_email_address,
other_description,

CASE
WHEN delivery_type = 'COURIER' THEN 'Courier'
WHEN delivery_type = 'HAND_DELIVER' THEN 'Self Drop-Off'
ELSE 'Other'
END AS delivery_method,


CASE
when delivery_window_end < DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 'Overdue Shipment'
when delivery_window_end > DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 'Awaiting Arrival'
when delivery_window_end > accepted_at then 'Shipment Accepted Within Window'
when delivery_window_end < accepted_at then 'Shipment Accepted Outside Window'
END AS acceptance_status,


case 
when total_quantity_delivered <> total_quantity_expected then 'Incorrect Quantity Submitted'
when total_quantity_delivered = total_quantity_expected then 'Correct Quantity Submitted'
when total_quantity_delivered is null and accepted_at is null then 'Not Accepted'
when total_quantity_delivered is null and accepted_at is not null then 'Not Counted'
END AS submission_accuracy


,CASE 
when delivery_window_end < DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 1 else 0
end as is_overdue
,case
when tracking_id is not null and tracking_id <> '' then concat('https://www.google.com/search?q=',tracking_id) 
end as tracking_link

,(
CASE
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when  cast(facility.facility_uuid as VARCHAR ) = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when  cast(facility.facility_uuid as VARCHAR ) = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when  cast(facility.facility_uuid as VARCHAR ) = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   
   else facility.name

  
END
) AS facility_sf_name


,
(
CASE
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        cast(facility.facility_uuid as VARCHAR ) IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
         cast(facility.facility_uuid as VARCHAR ) IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
        
        else 'other city'
END
) AS city,

(CASE 

when dc.country='Brazil' then 'Brazil'

when dc.country = 'United States' then 'US/CAN'

else 'SSL'

end) as subregion

,(case when facility.country_code in ('MX','CO','BR') then 'LATAM'
        when facility.country_code in ('US') then 'US/CAN'
         end ) as region


  
  
,photo_url, photo_file_id, qr_code,
source,	status,	published_at,
published_by_user_uuid,	marked_delivered_by_user_uuid,

cast(AT_TIMEZONE(receiving_started, facility.timezone) as TIMESTAMP ) as receiving_started_local,	


receiving_started_by_user_uuid,	arrived_space_uuid,
arrived_notes,
deliverer_designation,	deletion_reason, deletion_reason_notes


,date_diff('minute',sm.delivered_at,sm.receiving_started) as diff_received_delivered_min
,sm.receiving_started-sm.delivered_at as time_received_delivered


,date_diff('hour',sm.created_at,sm.delivered_at) as diff_created_delivered_hr
,sm.delivered_at-sm.created_at as time_created_delivered


,date_diff('minute',sm.receiving_started,sm.accepted_at) as diff_received_accepted_min
,sm.accepted_at-sm.receiving_started as time_accepted_receiving


,date_diff('minute',sm.delivered_at,sm.accepted_at) as diff_delivered_accepted_min
,sm.accepted_at-sm.delivered_at as time_delivered_accepted

,concat('https://stock.cloudkitchens.com/scanner/facility/',cast(facility.facility_uuid as VARCHAR ),'/receive-stock/',cast(uuid as varchar)) as sm_link




FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest as sm

left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON sm.facility_uuid=cast(facility.facility_uuid as VARCHAR)


left join hudi_ingest.processed.store_config_organization_view as org
on cast(sm.org_uuid as VARCHAR) =org.organization_id

left join shipment_line_item on 
cast(sm.uuid as VARCHAR ) = cast(shipment_line_item.shipment_manifest_uuid as VARCHAR )

JOIN hudi_ingest.scratch.dim_countries dc 
   ON facility.country_code = dc.country_code 
   --AND region = 'LATAM'


where sm.day_partition>='2020-01-01'
and facility.day_partition>='2000-01-01'
and date_trunc('day',date(sm.day_partition)) >= DATE_ADD('Day' ,-300, current_date)
and facility.country_code in ('BR','MX','CO','US'--,'EC','PE','PR','CL','GT','CR','DO','SV'
)
  

"""

print("querying ...")
por_sm = ds_trino.fetch_data(conn=conn, sql_str=query_por_sm, use_cache=False)
print("pulled")

querying ...
pulled


In [68]:
por_sm

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_uuid,org_uuid,accepted_at,created_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link
0,23070c53-9b9a-4c7c-8da8-592e0ac5137f,5,0,5,0,0,5,648,647.0,f0029277-0e7e-410a-ad4e-5247b892a280,bbbe0a77-0edb-4b65-b096-72ec19a3fcef,2023-07-18 23:07:54.876,2023-07-18 21:15:40.470,2023-07-18 14:15:40.470,2023-07-18 16:07:54.876,2023-07-18 23:03:01.066,2023-07-18 16:03:01.066,None,None,2023-07-19 21:00:00.000,2023-07-19 14:00:00.000,2023-07-19 23:00:00.000,2023-07-19 16:00:00.000,6b8399aa-d244-45bd-bd1d-81224372c04e,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,Brady's Bakery,US,United States,2023-07-18,2023-07-01,2023-07-16,,1,0,1,1,1,True,False,False,COURIER,Metrobi,,None,None,None,Courier,Shipment Accepted Within Window,Incorrect Quantity Submitted,0,None,NA-US-CA(S)-LA-Hollywood,other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,5dc9433a-2f9d-4376-8231-69df72d6c3ce,2023-07-18 16:03:01.066,5dc9433a-2f9d-4376-8231-69df72d6c3ce,None,,None,UNKNOWN,,0.0,0 00:00:00.000,1.0,0 01:47:20.596,4.0,0 00:04:53.810,4.0,0 00:04:53.810,https://stock.cloudkitchens.com/scanner/facili...
1,41b3b907-4da4-4517-bda4-3701d044d34a,7,0,7,6,4,7,4,4.0,76a6ef30-ddb2-4178-8c37-86c0d10db4b3,fcb693fe-7e55-4ae6-8ebe-c309cc1bb890,2023-03-17 15:47:05.894,2023-03-17 14:48:27.155,2023-03-17 09:48:27.155,2023-03-17 10:47:05.894,2023-03-17 15:41:46.451,2023-03-17 10:41:46.451,None,None,2023-03-17 15:00:00.000,2023-03-17 10:00:00.000,2023-03-17 15:30:00.000,2023-03-17 10:30:00.000,953a496f-a5ab-4317-b385-c8efe4861bec,SA-CO-Bogotá-Atabanza,America/Bogota,Waft Bakery,CO,Colombia,2023-03-17,2023-03-01,2023-03-12,,1,0,1,1,1,True,False,False,COURIER,waft,,None,None,None,Courier,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,BOG - Atabanza,Bogota,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,1fc6c141-81e7-429c-bbd1-78b2a1797051,2023-03-17 10:41:46.451,1fc6c141-81e7-429c-bbd1-78b2a1797051,None,,None,UNKNOWN,,0.0,0 00:00:00.000,0.0,0 00:53:19.296,5.0,0 00:05:19.443,5.0,0 00:05:19.443,https://stock.cloudkitchens.com/scanner/facili...
2,60f3ea9b-9415-40da-91f4-b9ff13c2fade,4,0,4,0,0,4,486,502.0,f0029277-0e7e-410a-ad4e-5247b892a280,bbbe0a77-0edb-4b65-b096-72ec19a3fcef,2023-03-17 22:09:47.871,2023-03-17 21:05:34.090,2023-03-17 14:05:34.090,2023-03-17 15:09:47.871,2023-03-17 22:06:06.000,2023-03-17 15:06:06.000,None,None,2023-03-17 21:30:00.000,2023-03-17 14:30:00.000,2023-03-17 22:30:00.000,2023-03-17 15:30:00.000,455ada76-83d9-4e21-937b-e03ff724ba7a,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,Brady's Bakery,US,United States,2023-03-17,2023-03-01,2023-03-12,,1,0,1,1,1,True,False,False,COURIER,,,No

In [69]:
types_sm=[]
for i in por_sm['comment']:
    try:
        j=ast.literal_eval(i)
        #print(j)
        tipos_sm=j.get('productTypes')
        #print(tipos, '\n')
        if len(tipos)==1:
            types_sm.append(tipos[0])
        else:
            types_sm.append(tipos_sm)
    except:
        types_sm.append(i)
        #print(i)
    

In [70]:
por_sm['type']=types_sm

In [71]:
por_sm['type'].value_counts()

                                        34086
1.0                                        71
2.0                                        35
3.0                                        30
4.0                                        29
                                        ...  
e83f9147-cbe6-442c-bc94-d4acbd37cbb9        1
8ffd3c81-0dc1-4905-8566-58a5286e36a3        1
94fb6130-1de3-4e44-a1ee-8f6cbfed5011        1
8fa1945a-7439-4bf7-8437-5605e539db8e        1
TIJ080823                                   1
Name: type, Length: 281, dtype: int64

In [72]:
por_sm

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_uuid,org_uuid,accepted_at,created_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link,type
0,23070c53-9b9a-4c7c-8da8-592e0ac5137f,5,0,5,0,0,5,648,647.0,f0029277-0e7e-410a-ad4e-5247b892a280,bbbe0a77-0edb-4b65-b096-72ec19a3fcef,2023-07-18 23:07:54.876,2023-07-18 21:15:40.470,2023-07-18 14:15:40.470,2023-07-18 16:07:54.876,2023-07-18 23:03:01.066,2023-07-18 16:03:01.066,None,None,2023-07-19 21:00:00.000,2023-07-19 14:00:00.000,2023-07-19 23:00:00.000,2023-07-19 16:00:00.000,6b8399aa-d244-45bd-bd1d-81224372c04e,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,Brady's Bakery,US,United States,2023-07-18,2023-07-01,2023-07-16,,1,0,1,1,1,True,False,False,COURIER,Metrobi,,None,None,None,Courier,Shipment Accepted Within Window,Incorrect Quantity Submitted,0,None,NA-US-CA(S)-LA-Hollywood,other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,5dc9433a-2f9d-4376-8231-69df72d6c3ce,2023-07-18 16:03:01.066,5dc9433a-2f9d-4376-8231-69df72d6c3ce,None,,None,UNKNOWN,,0.0,0 00:00:00.000,1.0,0 01:47:20.596,4.0,0 00:04:53.810,4.0,0 00:04:53.810,https://stock.cloudkitchens.com/scanner/facili...,
1,41b3b907-4da4-4517-bda4-3701d044d34a,7,0,7,6,4,7,4,4.0,76a6ef30-ddb2-4178-8c37-86c0d10db4b3,fcb693fe-7e55-4ae6-8ebe-c309cc1bb890,2023-03-17 15:47:05.894,2023-03-17 14:48:27.155,2023-03-17 09:48:27.155,2023-03-17 10:47:05.894,2023-03-17 15:41:46.451,2023-03-17 10:41:46.451,None,None,2023-03-17 15:00:00.000,2023-03-17 10:00:00.000,2023-03-17 15:30:00.000,2023-03-17 10:30:00.000,953a496f-a5ab-4317-b385-c8efe4861bec,SA-CO-Bogotá-Atabanza,America/Bogota,Waft Bakery,CO,Colombia,2023-03-17,2023-03-01,2023-03-12,,1,0,1,1,1,True,False,False,COURIER,waft,,None,None,None,Courier,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,BOG - Atabanza,Bogota,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,1fc6c141-81e7-429c-bbd1-78b2a1797051,2023-03-17 10:41:46.451,1fc6c141-81e7-429c-bbd1-78b2a1797051,None,,None,UNKNOWN,,0.0,0 00:00:00.000,0.0,0 00:53:19.296,5.0,0 00:05:19.443,5.0,0 00:05:19.443,https://stock.cloudkitchens.com/scanner/facili...,
2,60f3ea9b-9415-40da-91f4-b9ff13c2fade,4,0,4,0,0,4,486,502.0,f0029277-0e7e-410a-ad4e-5247b892a280,bbbe0a77-0edb-4b65-b096-72ec19a3fcef,2023-03-17 22:09:47.871,2023-03-17 21:05:34.090,2023-03-17 14:05:34.090,2023-03-17 15:09:47.871,2023-03-17 22:06:06.000,2023-03-17 15:06:06.000,None,None,2023-03-17 21:30:00.000,2023-03-17 14:30:00.000,2023-03-17 22:30:00.000,2023-03-17 15:30:00.000,455ada76-83d9-4e21-937b-e03ff724ba7a,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,Brady's Bakery,US,United States,2023-03-17,2023-03-01,2023-03-12,,1,0,1,1,1,True,False,False,COURI

In [73]:
por_sm.dtypes

shipment_manifest_uuid          object
num_skus                         int64
non_counted_sku                  int64
counted_sku                      int64
has_expiration_date              int64
                                ...   
time_accepted_receiving         object
diff_delivered_accepted_min    float64
time_delivered_accepted         object
sm_link                         object
type                            object
Length: 81, dtype: object

In [74]:
por_sm['delivery_window_start']=pd.to_datetime(por_sm['delivery_window_start'],errors='coerce')
por_sm['delivery_window_end']=pd.to_datetime(por_sm['delivery_window_end'],errors='coerce')
por_sm['accepted_at']=pd.to_datetime(por_sm['accepted_at'],errors='coerce')
por_sm['delivered_at']=pd.to_datetime(por_sm['delivered_at'],errors='coerce')

In [75]:
por_sm['delivery_windows_start_local']=pd.to_datetime(por_sm['delivery_windows_start_local'],errors='coerce')

In [76]:
por_sm['delivery_windows_end_local']=pd.to_datetime(por_sm['delivery_windows_end_local'],errors='coerce')


In [77]:
por_sm['accepeted_at_local']=pd.to_datetime(por_sm['accepeted_at_local'],errors='coerce')

In [78]:
por_sm['delivered_at_local']=pd.to_datetime(por_sm['delivered_at_local'],errors='coerce')

In [79]:
#https://stackoverflow.com/questions/10747974/how-to-check-if-the-current-time-is-in-range-in-python

def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end

In [80]:
delivered_promised=[]
for i in range(len(por_sm)):
    if pd.isnull(por_sm['delivered_at_local'][i]):
        valor='not_accepted'
    else:
        start=por_sm['delivery_windows_start_local'][i]
        end=por_sm['delivery_windows_end_local'][i]
        x=por_sm['delivered_at_local'][i]
    
        valor=time_in_range(start,end, x)
    
    delivered_promised.append(valor)
    

In [81]:
por_sm['delivered_in_the_promissed_range']=delivered_promised

In [82]:
por_sm['delivered_in_the_promissed_range'].value_counts()

False           19076
True            15101
not_accepted      725
Name: delivered_in_the_promissed_range, dtype: int64

In [83]:
def is_between(time_range_cliente, time_range_mañana,time_range_tarde):
    valor= ((time_range_cliente[0]>=time_range_mañana[0]) and (time_range_cliente[1]<time_range_mañana[1]))or((time_range_cliente[0]>=time_range_tarde[0]) and (time_range_cliente[1]<time_range_tarde[1]))
    return valor       



In [84]:
cliente=[por_sm['delivery_window_start'].dt.time[0],por_sm['delivery_window_end'].dt.time[0]]

In [85]:
cliente

[datetime.time(21, 0), datetime.time(23, 0)]

In [86]:
import numpy as np
import datetime

In [87]:
facility=[datetime.time(7,0,0),datetime.time(11,0,0)]

In [88]:
tarde=[datetime.time(15,0,0),datetime.time(17,0,0)]

In [89]:
tarde

[datetime.time(15, 0), datetime.time(17, 0)]

In [90]:
delivered_horario=[]
for i in range(len(por_sm)):
    
    rango_cliente_1=por_sm['delivery_windows_start_local'].dt.time[i]
    rango_cliente_2= por_sm['delivery_windows_end_local'].dt.time[i]
    mañana_1=datetime.time(7,0,0)
    mañana_2=datetime.time(11,0,0)
    tarde_1=datetime.time(15,0,0)
    tarde_2=datetime.time(17,0,0)
    
    valor=time_in_range(mañana_1,mañana_2,rango_cliente_1) or time_in_range(tarde_1,tarde_2,rango_cliente_2)
    delivered_horario.append(valor)
    if i%100==0:
        print('llevo ',i, 'iteraciones')

llevo  0 iteraciones
llevo  100 iteraciones
llevo  200 iteraciones
llevo  300 iteraciones
llevo  400 iteraciones
llevo  500 iteraciones
llevo  600 iteraciones
llevo  700 iteraciones
llevo  800 iteraciones
llevo  900 iteraciones
llevo  1000 iteraciones
llevo  1100 iteraciones
llevo  1200 iteraciones
llevo  1300 iteraciones
llevo  1400 iteraciones
llevo  1500 iteraciones
llevo  1600 iteraciones
llevo  1700 iteraciones
llevo  1800 iteraciones
llevo  1900 iteraciones
llevo  2000 iteraciones
llevo  2100 iteraciones
llevo  2200 iteraciones
llevo  2300 iteraciones
llevo  2400 iteraciones
llevo  2500 iteraciones
llevo  2600 iteraciones
llevo  2700 iteraciones
llevo  2800 iteraciones
llevo  2900 iteraciones
llevo  3000 iteraciones
llevo  3100 iteraciones
llevo  3200 iteraciones
llevo  3300 iteraciones
llevo  3400 iteraciones
llevo  3500 iteraciones
llevo  3600 iteraciones
llevo  3700 iteraciones
llevo  3800 iteraciones
llevo  3900 iteraciones
llevo  4000 iteraciones
llevo  4100 iteraciones
llev

In [91]:
por_sm['programacion_rango_facility']=delivered_horario

In [92]:
def horario_facility(entrega_cliente, time_range_mañana,time_range_tarde):
    valor= ((entrega_cliente>=time_range_mañana[0]) and (entrega_cliente<time_range_mañana[1]))or((entrega_cliente>=time_range_tarde[0]) and (entrega_cliente<time_range_tarde[1]))
    return valor

In [93]:
mañana=[datetime.time(7,0,0),datetime.time(11,0,0)]
tarde=[datetime.time(15,0,0),datetime.time(17,0,0)]
ejemplo=por_sm['delivered_at'].dt.time[300]

In [94]:
ejemplo

datetime.time(12, 40, 27)

In [95]:
horario_facility(ejemplo,mañana,tarde)

False

In [96]:
mañana_1=datetime.time(7,0,0)
mañana_2=datetime.time(11,0,0)
tarde_1=datetime.time(15,0,0)
tarde_2=datetime.time(17,0,0)
x=por_sm['delivered_at'].dt.time[500]

In [97]:
print(type(mañana_1),mañana_2,type(x))

<class 'datetime.time'> 11:00:00 <class 'datetime.time'>


In [98]:
por_sm['delivered_at_local'].isnull().sum()

725

In [99]:
#entrego en el horario de la facility
delivered_horario_facility=[]
for i in range(len(por_sm)):
    mañana=[datetime.time(7,0,0),datetime.time(11,0,0)]
    tarde=[datetime.time(15,0,0),datetime.time(17,0,0)]
    ejemplo=por_sm['delivered_at_local'].dt.time[i]
    
    try:
        valor=horario_facility(ejemplo,mañana,tarde)
    except:
        valor='not accepted'
    
    delivered_horario_facility.append(valor)
    
    
    
    

In [100]:
por_sm['entrega_dentro_rango_facility']=delivered_horario_facility

In [101]:
por_sm['delivery_window_end'].dt.date[1]==por_sm['delivered_at'].dt.date[1]

True

In [102]:
pd.isnull(por_sm['delivered_at'][10])

False

In [103]:
por_sm[por_sm['delivered_at_local'].isnull()]

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_uuid,org_uuid,accepted_at,created_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link,type,delivered_in_the_promissed_range,programacion_rango_facility,entrega_dentro_rango_facility
27,96e84ddf-ea69-48c0-a974-486b4d57a2c3,14,14,0,14,0,0,138,NaN,5315a66f-8337-4c68-af42-86089a86f8ba,4bc91e6c-a3b6-40da-9ed6-7cd15b617de4,NaT,2023-04-28 16:24:43.986,2023-04-28 09:24:43.986,NaT,NaT,NaT,2023-05-08 20:56:48.373,2023-05-08 13:56:48.373,2023-05-02 19:00:00,2023-05-02 12:00:00,2023-05-02 22:00:00,2023-05-02 15:00:00,None,NA-US-CA(S)-LA-LongBeach (Correct),America/Los_Angeles,Hey Sugar LLC,US,United States,2023-04-28,2023-04-01,2023-04-23,,0,1,1,0,0,False,True,False,HAND_DELIVER,None,None,Chan Buie,chan_buie@heysugarsweets.com,None,Self Drop-Off,Overdue Shipment,Not Accepted,1,None,NA-US-CA(S)-LA-LongBeach (Correct),other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,DELETED,None,None,None,None,None,None,,DELIVERER_DESIGNATION_OWNER,SHIPMENT_NEVER_ARRIVED,,NaN,None,NaN,None,NaN,None,NaN,None,https://stock.cloudkitchens.com/scanner/facili...,,not_accepted,True,not accepted
29,9f81dafd-dc04-48bf-a74e-c2b2ba0d3ba0,4,4,0,0,0,0,132,NaN,0ee5dccf-ed5b-465c-9858-b53351353e3b,be169c79-231d-48cd-81b1-23800c573374,NaT,2022-12-27 17:33:11.211,2022-12-27 14:33:11.211,NaT,NaT,NaT,2023-01-24 12:49:10.451,2023-01-24 09:49:10.451,2022-12-27 18:00:00,2022-12-27 15:00:00,2022-12-27 20:00:00,2022-12-27 17:00:00,None,SA-BR-RJ-Camerino,America/Sao_Paulo,Los Paleteros,BR,Brazil,2022-12-27,2022-12-01,2022-12-25,,0,1,1,0,0,False,False,True,HAND_DELIVER,None,None,Marcio,logistica.curitiba@lospaleteros.com.br,None,Self Drop-Off,Overdue Shipment,Not Accepted,1,None,RIO - Camerino,Rio de Janeiro,Brazil,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,DELETED,None,None,None,None,None,None,,DELIVERER_DESIGNATION_EMPLOYEE,UNKNOWN,,NaN,None,NaN,None,NaN,None,NaN,None,https://stock.cloudkitchens.com/scanner/facili...,,not_accepted,True,not accepted
36,7e95eb92-b49d-4629-8e48-a786628a079c,7,7,0,0,0,0,80,NaN,305dca7a-ffcf-4ecf-9363-4238af3cba6b,07d8cf64-c8f9-4cd8-978b-a0dac7aed733,NaT,2022-12-30 23:10:38.260,2022-12-30 18:10:38.260,NaT,NaT,NaT,2023-02-13 22:48:27.317,2023-02-13 17:48:27.317,2022-12-31 14:00:00,2022-12-31 09:00:00,2022-12-31 22:00:00,2022-12-31 17:00:00,None,NA-US-PA-PHL-Fairmount,America/New_York,Oil And Fire Group LLC,US,United States,2022-12-30,2022-12-01,2022-12-25,,0,1,1,0,0,False,True,False,COURIER,USPS,,None,None,None,Courier,Overdue Shipment,Not Accepted,1,None,NA-US-PA-PHL-Fairmount,other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_UNKNOWN,DELETED,None,None,None,None,None,None,,None

In [104]:
por_sm.columns

Index(['shipment_manifest_uuid', 'num_skus', 'non_counted_sku', 'counted_sku',
       'has_expiration_date', 'has_space_uuid', 'has_live_at',
       'total_quantity_expected', 'total_quantity_delivered', 'facility_uuid',
       'org_uuid', 'accepted_at', 'created_at', 'created_at_local',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'deleted_sm_date', 'deleted_at_local', 'delivery_window_start',
       'delivery_windows_start_local', 'delivery_window_end',
       'delivery_windows_end_local', 'transaction_uuid', 'facility_name',
       'timezone', 'org_name', 'country_code', 'country', 'created_day',
       'created_month', 'created_week', 'comment', 'accepted_num',
       'deleted_num', 'created_num', 'receiving_started_num', 'delivered_num',
       'contains_dry_product_type', 'contains_cold_product_type',
       'contains_frozen_product_type', 'delivery_type', 'company_name',
       'tracking_id', 'deliverer_name', 'deliverer_email_address',
       'other

In [105]:
delivered_día_prometido=[]
for i in range(len(por_sm)):
    if pd.isnull(por_sm['delivered_at_local'][i]):
        valor='not_accepted'
    else:
        valor=por_sm['delivery_windows_end_local'].dt.date[i]==por_sm['delivered_at_local'].dt.date[i]
    delivered_día_prometido.append(valor)

In [106]:
por_sm['delivered_at_promised_day']=delivered_día_prometido

In [107]:
por_sm

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_uuid,org_uuid,accepted_at,created_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link,type,delivered_in_the_promissed_range,programacion_rango_facility,entrega_dentro_rango_facility,delivered_at_promised_day
0,23070c53-9b9a-4c7c-8da8-592e0ac5137f,5,0,5,0,0,5,648,647.0,f0029277-0e7e-410a-ad4e-5247b892a280,bbbe0a77-0edb-4b65-b096-72ec19a3fcef,2023-07-18 23:07:54.876,2023-07-18 21:15:40.470,2023-07-18 14:15:40.470,2023-07-18 16:07:54.876,2023-07-18 23:03:01.066,2023-07-18 16:03:01.066,None,None,2023-07-19 21:00:00,2023-07-19 14:00:00,2023-07-19 23:00:00,2023-07-19 16:00:00,6b8399aa-d244-45bd-bd1d-81224372c04e,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,Brady's Bakery,US,United States,2023-07-18,2023-07-01,2023-07-16,,1,0,1,1,1,True,False,False,COURIER,Metrobi,,None,None,None,Courier,Shipment Accepted Within Window,Incorrect Quantity Submitted,0,None,NA-US-CA(S)-LA-Hollywood,other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,5dc9433a-2f9d-4376-8231-69df72d6c3ce,2023-07-18 16:03:01.066,5dc9433a-2f9d-4376-8231-69df72d6c3ce,None,,None,UNKNOWN,,0.0,0 00:00:00.000,1.0,0 01:47:20.596,4.0,0 00:04:53.810,4.0,0 00:04:53.810,https://stock.cloudkitchens.com/scanner/facili...,,False,True,True,False
1,41b3b907-4da4-4517-bda4-3701d044d34a,7,0,7,6,4,7,4,4.0,76a6ef30-ddb2-4178-8c37-86c0d10db4b3,fcb693fe-7e55-4ae6-8ebe-c309cc1bb890,2023-03-17 15:47:05.894,2023-03-17 14:48:27.155,2023-03-17 09:48:27.155,2023-03-17 10:47:05.894,2023-03-17 15:41:46.451,2023-03-17 10:41:46.451,None,None,2023-03-17 15:00:00,2023-03-17 10:00:00,2023-03-17 15:30:00,2023-03-17 10:30:00,953a496f-a5ab-4317-b385-c8efe4861bec,SA-CO-Bogotá-Atabanza,America/Bogota,Waft Bakery,CO,Colombia,2023-03-17,2023-03-01,2023-03-12,,1,0,1,1,1,True,False,False,COURIER,waft,,None,None,None,Courier,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,BOG - Atabanza,Bogota,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,1fc6c141-81e7-429c-bbd1-78b2a1797051,2023-03-17 10:41:46.451,1fc6c141-81e7-429c-bbd1-78b2a1797051,None,,None,UNKNOWN,,0.0,0 00:00:00.000,0.0,0 00:53:19.296,5.0,0 00:05:19.443,5.0,0 00:05:19.443,https://stock.cloudkitchens.com/scanner/facili...,,False,True,True,True
2,60f3ea9b-9415-40da-91f4-b9ff13c2fade,4,0,4,0,0,4,486,502.0,f0029277-0e7e-410a-ad4e-5247b892a280,bbbe0a77-0edb-4b65-b096-72ec19a3fcef,2023-03-17 22:09:47.871,2023-03-17 21:05:34.090,2023-03-17 14:05:34.090,2023-03-17 15:09:47.871,2023-03-17 22:06:06.000,2023-03-17 15:06:06.000,None,None,2023-03-17 21:30:00,2023-03-17 14:30:00,2023-03-17 22:30:00,2023-03-17 15:30:00,455ada76-83d9-4e21-937b-e03ff724ba7a,NA-US-CA(S)-LA-Hollywood,Americ

In [108]:
por_sm['overdue']=np.where(por_sm['delivered_at'].dt.date==por_sm['accepted_at'].dt.date,0,1)

In [109]:
por_sm['overdue'].value_counts()

0    31527
1     3375
Name: overdue, dtype: int64

In [110]:
por_sm['is_overdue'].value_counts()

0    34312
1      590
Name: is_overdue, dtype: int64

In [111]:
por_sm['created_day']=pd.to_datetime(por_sm['created_day'])

In [112]:
#por_sm['created_at_local']=pd.to_datetime('created_at_local', errors='coerce')

In [113]:
#por_sm.columns

In [114]:
#por_sm['created_at_local']

In [115]:
por_sm.columns

Index(['shipment_manifest_uuid', 'num_skus', 'non_counted_sku', 'counted_sku',
       'has_expiration_date', 'has_space_uuid', 'has_live_at',
       'total_quantity_expected', 'total_quantity_delivered', 'facility_uuid',
       'org_uuid', 'accepted_at', 'created_at', 'created_at_local',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'deleted_sm_date', 'deleted_at_local', 'delivery_window_start',
       'delivery_windows_start_local', 'delivery_window_end',
       'delivery_windows_end_local', 'transaction_uuid', 'facility_name',
       'timezone', 'org_name', 'country_code', 'country', 'created_day',
       'created_month', 'created_week', 'comment', 'accepted_num',
       'deleted_num', 'created_num', 'receiving_started_num', 'delivered_num',
       'contains_dry_product_type', 'contains_cold_product_type',
       'contains_frozen_product_type', 'delivery_type', 'company_name',
       'tracking_id', 'deliverer_name', 'deliverer_email_address',
       'other

In [116]:
por_sm['created_at']=pd.to_datetime(por_sm['created_at'],errors='coerce')

In [117]:
por_sm['accepted_time_delta']=por_sm['accepted_at']-por_sm['created_at']

In [118]:
por_sm['accepted_time_delta']

0       0 days 01:52:14.406000
1       0 days 00:58:38.739000
2       0 days 01:04:13.781000
3       0 days 01:51:00.437000
4       0 days 12:33:20.382000
                 ...          
34897   0 days 02:11:01.610000
34898   0 days 00:00:54.032000
34899   2 days 04:01:12.386000
34900   1 days 00:18:09.128000
34901   0 days 19:33:55.908000
Name: accepted_time_delta, Length: 34902, dtype: timedelta64[ns]

In [119]:
por_sm['accepted_time_hr']=por_sm['accepted_time_delta'].dt.components['days']*24+por_sm['accepted_time_delta'].dt.components['hours']+por_sm['accepted_time_delta'].dt.components['minutes']/60+por_sm['accepted_time_delta'].dt.components['seconds']/3600

In [120]:
por_sm['accepted_time_hr']

0         1.870556
1         0.977222
2         1.070278
3         1.850000
4        12.555556
           ...    
34897     2.183611
34898     0.015000
34899    52.020000
34900    24.302500
34901    19.565278
Name: accepted_time_hr, Length: 34902, dtype: float64

In [121]:
por_sm['received_time_delta']=por_sm['accepted_at']-por_sm['delivered_at']

In [122]:
por_sm['received_time_hr']=por_sm['received_time_delta'].dt.components['days']/24+por_sm['received_time_delta'].dt.components['hours']+por_sm['received_time_delta'].dt.components['minutes']/60+por_sm['received_time_delta'].dt.components['seconds']/3600

In [123]:
por_sm['received_time_less_2hr']=np.where(por_sm['received_time_hr']<2,1,0)

In [124]:
por_sm['received_time_less_2hr'].value_counts()

1    28976
0     5926
Name: received_time_less_2hr, dtype: int64

In [125]:
por_sm['received_time_2hr_8hr']=np.where((por_sm['received_time_hr']>=2)&(por_sm['received_time_hr']<=8),1,0)

In [126]:
por_sm['received_time_2hr_8hr'].value_counts()

0    31732
1     3170
Name: received_time_2hr_8hr, dtype: int64

In [127]:
por_sm['received_time_8hr_24hr']=np.where((por_sm['received_time_hr']>=8)&(por_sm['received_time_hr']<=24),1,0)

In [128]:
por_sm['received_time_8hr_24hr'].value_counts()

0    32936
1     1966
Name: received_time_8hr_24hr, dtype: int64

In [129]:
por_sm['received_time_more_24hr']=np.where((por_sm['received_time_hr']>24),1,0)

In [130]:
por_sm['received_time_more_24hr'].value_counts()

0    34891
1       11
Name: received_time_more_24hr, dtype: int64

In [131]:
por_sm['Counting_compliance']=np.where(por_sm['counted_sku']>0,1,0)

In [132]:
por_sm['Counting_compliance'].value_counts()

1    33493
0     1409
Name: Counting_compliance, dtype: int64

In [133]:
por_sm['Count_expirations']=np.where(por_sm['has_expiration_date']>0,1,0)

In [134]:
por_sm['Count_expirations'].value_counts()

0    19879
1    15023
Name: Count_expirations, dtype: int64

In [135]:
por_sm['Count_space_tracking']=np.where(por_sm['has_space_uuid']>0,1,0)

In [136]:
por_sm['Shipment_with_Set_live_SKUs']=np.where((por_sm['has_live_at']>0)&(por_sm['accepted_num']==1),1,0)

In [137]:
por_sm['Shipment_with_Set_live_SKUs'].value_counts()

1    34123
0      779
Name: Shipment_with_Set_live_SKUs, dtype: int64

In [138]:
por_sm.columns

Index(['shipment_manifest_uuid', 'num_skus', 'non_counted_sku', 'counted_sku',
       'has_expiration_date', 'has_space_uuid', 'has_live_at',
       'total_quantity_expected', 'total_quantity_delivered', 'facility_uuid',
       'org_uuid', 'accepted_at', 'created_at', 'created_at_local',
       'accepeted_at_local', 'delivered_at', 'delivered_at_local',
       'deleted_sm_date', 'deleted_at_local', 'delivery_window_start',
       'delivery_windows_start_local', 'delivery_window_end',
       'delivery_windows_end_local', 'transaction_uuid', 'facility_name',
       'timezone', 'org_name', 'country_code', 'country', 'created_day',
       'created_month', 'created_week', 'comment', 'accepted_num',
       'deleted_num', 'created_num', 'receiving_started_num', 'delivered_num',
       'contains_dry_product_type', 'contains_cold_product_type',
       'contains_frozen_product_type', 'delivery_type', 'company_name',
       'tracking_id', 'deliverer_name', 'deliverer_email_address',
       'other

In [139]:
def tipo (df):
    lista=[]
    for i in range(len(df)):
        if (df['contains_cold_product_type'][i]==True) and ((df['contains_dry_product_type'][i]==True) or (df['contains_frozen_product_type'][i]==True)):
            valor ='Mix'
        elif (df['contains_dry_product_type'][i]==True) and ((df['contains_cold_product_type'][i]==True) or (df['contains_frozen_product_type'][i]==True)):
            valor ='Mix'
        elif (df['contains_frozen_product_type'][i]==True) and ((df['contains_dry_product_type'][i]==True) or (df['contains_cold_product_type'][i]==True)):
            valor ='Mix'
        elif df['contains_cold_product_type'][i]==True:
            valor='Cold'
        elif df['contains_dry_product_type'][i]==True:
            valor='Dry'
        elif df['contains_frozen_product_type'][i]==True:
            valor='Frozen'
        else:
            valor='Unknown'
        
        lista.append(valor)
    return lista

In [140]:
nuevo_tipo_lista=tipo(por_sm)

In [141]:
por_sm['Temp_type']=nuevo_tipo_lista

In [142]:
por_sm['Temp_type'].value_counts()

Cold       12078
Frozen     11063
Dry         9149
Mix         2560
Unknown       52
Name: Temp_type, dtype: int64

In [143]:
por_sm['delivered_in_the_promissed_range_num']=np.where(por_sm['delivered_in_the_promissed_range']==True,1,0)

In [144]:
por_sm['delivered_in_the_promissed_range_num'].value_counts()

0    19801
1    15101
Name: delivered_in_the_promissed_range_num, dtype: int64

In [145]:
por_sm['programacion_rango_facility_num']=np.where(por_sm['programacion_rango_facility']==True,1,0)

In [146]:
por_sm['programacion_rango_facility_num'].value_counts()

1    32689
0     2213
Name: programacion_rango_facility_num, dtype: int64

In [147]:
por_sm['entrega_dentro_rango_facility_num']=np.where(por_sm['entrega_dentro_rango_facility']==True,1,0)

In [148]:
por_sm['entrega_dentro_rango_facility_num'].value_counts()

0    19193
1    15709
Name: entrega_dentro_rango_facility_num, dtype: int64

In [149]:
por_sm['delivered_at_promised_day_num']=np.where(por_sm['delivered_at_promised_day']==True,1,0)

In [150]:
por_sm['delivered_at_promised_day_num'].value_counts()

1    27355
0     7547
Name: delivered_at_promised_day_num, dtype: int64

In [151]:
por_sm_spread=por_sm[por_sm['created_day']>=fecha_dias].reset_index(drop=True)

In [152]:
por_sm_spread

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,facility_uuid,org_uuid,accepted_at,created_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,sm_link,type,delivered_in_the_promissed_range,programacion_rango_facility,entrega_dentro_rango_facility,delivered_at_promised_day,overdue,accepted_time_delta,accepted_time_hr,received_time_delta,received_time_hr,received_time_less_2hr,received_time_2hr_8hr,received_time_8hr_24hr,received_time_more_24hr,Counting_compliance,Count_expirations,Count_space_tracking,Shipment_with_Set_live_SKUs,Temp_type,delivered_in_the_promissed_range_num,programacion_rango_facility_num,entrega_dentro_rango_facility_num,delivered_at_promised_day_num
0,d4ea40b6-e206-4198-bb15-37bc57a3ec46,1,1,0,1,0,1,1,NaN,f0029277-0e7e-410a-ad4e-5247b892a280,55b8f823-f905-470f-95f3-73bccbb5cbbd,2023-08-11 23:49:19.416,2023-08-11 21:58:18.979,2023-08-11 14:58:18.979,2023-08-11 16:49:19.416,2023-08-11 22:00:19.000,2023-08-11 15:00:19.000,None,None,2023-08-12 17:00:00,2023-08-12 10:00:00,2023-08-12 17:15:00,2023-08-12 10:15:00,6ac5634d-dcb7-4697-b811-4aefacab2eb0,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,CrunchyChewy,US,United States,2023-08-11,2023-08-01,2023-08-06,,1,0,1,1,1,True,False,False,HAND_DELIVER,None,None,,,None,Self Drop-Off,Shipment Accepted Within Window,Not Counted,0,None,NA-US-CA(S)-LA-Hollywood,other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,5dc9433a-2f9d-4376-8231-69df72d6c3ce,2023-08-11 16:49:19.416,42d4de65-88a0-4993-91d4-4cd40b371c02,None,,DELIVERER_DESIGNATION_UNKNOWN,UNKNOWN,,109.0,0 01:49:00.416,0.0,0 00:02:00.021,0.0,0 00:00:00.000,109.0,0 01:49:00.416,https://stock.cloudkitchens.com/scanner/facili...,,False,True,True,False,0,0 days 01:51:00.437000,1.850000,0 days 01:49:00.416000,1.816667,1,0,0,0,0,1,0,1,Dry,0,1,1,0
1,49f8a217-ea4e-4433-ad9e-78e625008d8b,9,0,9,9,2,9,22,22.0,47e2d054-8b82-420c-b1b0-18010f57489c,e197f049-c172-48f8-a9f9-69e5e520b7a9,2023-08-19 18:20:41.032,2023-08-19 04:34:35.629,2023-08-18 22:34:35.629,2023-08-19 12:20:41.032,2023-08-19 15:42:07.228,2023-08-19 09:42:07.228,None,None,2023-08-19 15:00:00,2023-08-19 09:00:00,2023-08-19 17:00:00,2023-08-19 11:00:00,7b8a6104-f1e0-4f26-9e5e-b0ce15933ef0,NA-MX-CDMX-Insurgentes Sur,America/Mexico_City,Pluck Botanas,MX,Mexico,2023-08-19,2023-08-01,2023-08-13,,1,0,1,1,1,True,False,False,HAND_DELIVER,None,None,CARLOS GARCIA VILLARREAL,carlosgvcdmx@hotmail.com,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,MEX - Insurgentes Sur,Mexico City,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,2023-08-19 09:42:07.228,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,None,,DELIVERER_DE

In [153]:
por_sm_spread.columns

Index(['shipment_manifest_uuid', 'num_skus', 'non_counted_sku', 'counted_sku',
       'has_expiration_date', 'has_space_uuid', 'has_live_at',
       'total_quantity_expected', 'total_quantity_delivered', 'facility_uuid',
       ...
       'received_time_more_24hr', 'Counting_compliance', 'Count_expirations',
       'Count_space_tracking', 'Shipment_with_Set_live_SKUs', 'Temp_type',
       'delivered_in_the_promissed_range_num',
       'programacion_rango_facility_num', 'entrega_dentro_rango_facility_num',
       'delivered_at_promised_day_num'],
      dtype='object', length=103)

In [154]:
por_sm_spread_2=por_sm_spread[[#'sm_uuid',
'shipment_manifest_uuid','num_skus','non_counted_sku','counted_sku','has_expiration_date',
    'has_space_uuid','has_live_at','total_quantity_expected','total_quantity_delivered','org_uuid','facility_uuid',
    'accepted_at','created_at_local','accepeted_at_local','delivered_at','delivered_at_local',
    'deleted_sm_date','deleted_at_local','delivery_window_start','delivery_windows_start_local',
    'delivery_window_end','delivery_windows_end_local','transaction_uuid','facility_name','timezone',
    'org_name','country_code','country','created_day','created_month','created_week','comment','accepted_num',
    'deleted_num','created_num','receiving_started_num','delivered_num','contains_dry_product_type',
    'contains_cold_product_type','contains_frozen_product_type','delivery_type','company_name',
    'tracking_id','deliverer_name','deliverer_email_address','other_description','delivery_method',
    'acceptance_status','submission_accuracy','is_overdue','tracking_link','facility_sf_name','city',
    'subregion','region','photo_url','photo_file_id','qr_code','source','status','published_at',
    'published_by_user_uuid','marked_delivered_by_user_uuid','receiving_started_local',
    'receiving_started_by_user_uuid','arrived_space_uuid','arrived_notes','deliverer_designation',
    'deletion_reason','deletion_reason_notes','diff_received_delivered_min','time_received_delivered',
    'diff_created_delivered_hr','time_created_delivered','diff_received_accepted_min',
    'time_accepted_receiving','diff_delivered_accepted_min','time_delivered_accepted',
    
    'received_time_hr', 'received_time_less_2hr', 'received_time_2hr_8hr',
       'received_time_8hr_24hr', 'received_time_more_24hr',
        'accepted_time_hr','accepted_time_delta',                       
                               
       'Counting_compliance', 'Count_expirations', 'Count_space_tracking',
       'Shipment_with_Set_live_SKUs', 'Temp_type',
    'delivered_in_the_promissed_range_num',
       'programacion_rango_facility_num', 'entrega_dentro_rango_facility_num',
       'delivered_at_promised_day_num',
    
    'sm_link'
]]

In [155]:
por_sm_spread_2

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,org_uuid,facility_uuid,accepted_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,received_time_hr,received_time_less_2hr,received_time_2hr_8hr,received_time_8hr_24hr,received_time_more_24hr,accepted_time_hr,accepted_time_delta,Counting_compliance,Count_expirations,Count_space_tracking,Shipment_with_Set_live_SKUs,Temp_type,delivered_in_the_promissed_range_num,programacion_rango_facility_num,entrega_dentro_rango_facility_num,delivered_at_promised_day_num,sm_link
0,d4ea40b6-e206-4198-bb15-37bc57a3ec46,1,1,0,1,0,1,1,NaN,55b8f823-f905-470f-95f3-73bccbb5cbbd,f0029277-0e7e-410a-ad4e-5247b892a280,2023-08-11 23:49:19.416,2023-08-11 14:58:18.979,2023-08-11 16:49:19.416,2023-08-11 22:00:19.000,2023-08-11 15:00:19.000,None,None,2023-08-12 17:00:00,2023-08-12 10:00:00,2023-08-12 17:15:00,2023-08-12 10:15:00,6ac5634d-dcb7-4697-b811-4aefacab2eb0,NA-US-CA(S)-LA-Hollywood,America/Los_Angeles,CrunchyChewy,US,United States,2023-08-11,2023-08-01,2023-08-06,,1,0,1,1,1,True,False,False,HAND_DELIVER,None,None,,,None,Self Drop-Off,Shipment Accepted Within Window,Not Counted,0,None,NA-US-CA(S)-LA-Hollywood,other city,US/CAN,US/CAN,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,5dc9433a-2f9d-4376-8231-69df72d6c3ce,2023-08-11 16:49:19.416,42d4de65-88a0-4993-91d4-4cd40b371c02,None,,DELIVERER_DESIGNATION_UNKNOWN,UNKNOWN,,109.0,0 01:49:00.416,0.0,0 00:02:00.021,0.0,0 00:00:00.000,109.0,0 01:49:00.416,1.816667,1,0,0,0,1.850000,0 days 01:51:00.437000,0,1,0,1,Dry,0,1,1,0,https://stock.cloudkitchens.com/scanner/facili...
1,49f8a217-ea4e-4433-ad9e-78e625008d8b,9,0,9,9,2,9,22,22.0,e197f049-c172-48f8-a9f9-69e5e520b7a9,47e2d054-8b82-420c-b1b0-18010f57489c,2023-08-19 18:20:41.032,2023-08-18 22:34:35.629,2023-08-19 12:20:41.032,2023-08-19 15:42:07.228,2023-08-19 09:42:07.228,None,None,2023-08-19 15:00:00,2023-08-19 09:00:00,2023-08-19 17:00:00,2023-08-19 11:00:00,7b8a6104-f1e0-4f26-9e5e-b0ce15933ef0,NA-MX-CDMX-Insurgentes Sur,America/Mexico_City,Pluck Botanas,MX,Mexico,2023-08-19,2023-08-01,2023-08-13,,1,0,1,1,1,True,False,False,HAND_DELIVER,None,None,CARLOS GARCIA VILLARREAL,carlosgvcdmx@hotmail.com,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,MEX - Insurgentes Sur,Mexico City,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,2023-08-19 09:42:07.228,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,None,,DELIVERER_DESIGNATION_OWNER,UNKNOWN,,0.0,0 00:00:00.000,11.0,0 11:07:31.599,158.0,0 02:38:33.804,158.0,0 02:38:33.804,2.642500,0,1,0,0,13.768056,0 days 13:46:05.403000,1,1,1,1,Dry,1,1,1,1,https://stock.cloudkitchens.com/scanner/facili...
2,95ef250b-b9a8-4637-89f4-4ef38

In [156]:
por_sm_spread_latam=por_sm_spread_2[por_sm_spread_2['country'].isin(['Mexico','Colombia','Brazil'])].reset_index(drop=True)

In [157]:
por_sm_spread_latam

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,org_uuid,facility_uuid,accepted_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,received_time_hr,received_time_less_2hr,received_time_2hr_8hr,received_time_8hr_24hr,received_time_more_24hr,accepted_time_hr,accepted_time_delta,Counting_compliance,Count_expirations,Count_space_tracking,Shipment_with_Set_live_SKUs,Temp_type,delivered_in_the_promissed_range_num,programacion_rango_facility_num,entrega_dentro_rango_facility_num,delivered_at_promised_day_num,sm_link
0,49f8a217-ea4e-4433-ad9e-78e625008d8b,9,0,9,9,2,9,22,22.0,e197f049-c172-48f8-a9f9-69e5e520b7a9,47e2d054-8b82-420c-b1b0-18010f57489c,2023-08-19 18:20:41.032,2023-08-18 22:34:35.629,2023-08-19 12:20:41.032,2023-08-19 15:42:07.228,2023-08-19 09:42:07.228,None,None,2023-08-19 15:00:00,2023-08-19 09:00:00,2023-08-19 17:00:00,2023-08-19 11:00:00,7b8a6104-f1e0-4f26-9e5e-b0ce15933ef0,NA-MX-CDMX-Insurgentes Sur,America/Mexico_City,Pluck Botanas,MX,Mexico,2023-08-19,2023-08-01,2023-08-13,,1,0,1,1,1,True,False,False,HAND_DELIVER,None,None,CARLOS GARCIA VILLARREAL,carlosgvcdmx@hotmail.com,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,MEX - Insurgentes Sur,Mexico City,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,2023-08-19 09:42:07.228,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,None,,DELIVERER_DESIGNATION_OWNER,UNKNOWN,,0.0,0 00:00:00.000,11.0,0 11:07:31.599,158.0,0 02:38:33.804,158.0,0 02:38:33.804,2.642500,0,1,0,0,13.768056,0 days 13:46:05.403000,1,1,1,1,Dry,1,1,1,1,https://stock.cloudkitchens.com/scanner/facili...
1,95ef250b-b9a8-4637-89f4-4ef38d923f6f,31,2,29,31,15,31,64,62.0,93af458e-e1eb-4483-9a10-abd62e2343da,3759ad21-4537-48ea-b1f1-48809e8a03ac,2023-08-29 14:43:31.419,2023-08-25 16:48:40.725,2023-08-29 11:43:31.419,2023-08-28 20:00:53.000,2023-08-28 17:00:53.000,None,None,2023-08-28 18:00:00,2023-08-28 15:00:00,2023-08-28 20:00:00,2023-08-28 17:00:00,31bdaca9-4df8-4af9-ac97-de5f3c66c18d,SA-BR-RJ-Botafogo,America/Sao_Paulo,Moulin Brown - Pik n' Pak,BR,Brazil,2023-08-25,2023-08-01,2023-08-20,,1,0,1,1,1,True,False,False,PARCEL,PARCEL_CARRIER_OTHER,157132602827,None,None,None,Other,Shipment Accepted Outside Window,Incorrect Quantity Submitted,0,https://www.google.com/search?q=157132602827,RIO - Botafogo,Rio de Janeiro,Brazil,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_UNKNOWN,ACCEPTED,None,None,15a112ee-6fd6-46ba-8bdc-5b3e53baef98,2023-08-29 11:29:06.886,86433d41-85a6-4ee7-a770-d62accaf3de6,None,,None,UNKNOWN,,1108.0,0 18:28:13.886,72.0,3 00:12:12.275,14.0,0 00:14:24.533,1122.0,0 18:42:38.419,18.710556,0,0,1,0,90.913889,3 days 18:54:50.694000,1,1,1,1,Dry,0,1,0,1,https://stock.cloudkitchen

In [158]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1Ffh0lxSI31wEckOrbpB1gxgd4ry3GdXO62XvksepGds')

# select sheet

ws = ss.worksheet('title','data_por_sm')

print('Clearing Range ...')
ws.clear(start='A', end='co')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(por_sm_spread_latam, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [159]:
por_sm_spread_ssl=por_sm_spread_2[por_sm_spread_2['country'].isin(['Mexico','Colombia'])].reset_index(drop=True)

In [160]:
por_sm_spread_ssl

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,has_expiration_date,has_space_uuid,has_live_at,total_quantity_expected,total_quantity_delivered,org_uuid,facility_uuid,accepted_at,created_at_local,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted_num,deleted_num,created_num,receiving_started_num,delivered_num,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,city,subregion,region,photo_url,photo_file_id,qr_code,source,status,published_at,published_by_user_uuid,marked_delivered_by_user_uuid,receiving_started_local,receiving_started_by_user_uuid,arrived_space_uuid,arrived_notes,deliverer_designation,deletion_reason,deletion_reason_notes,diff_received_delivered_min,time_received_delivered,diff_created_delivered_hr,time_created_delivered,diff_received_accepted_min,time_accepted_receiving,diff_delivered_accepted_min,time_delivered_accepted,received_time_hr,received_time_less_2hr,received_time_2hr_8hr,received_time_8hr_24hr,received_time_more_24hr,accepted_time_hr,accepted_time_delta,Counting_compliance,Count_expirations,Count_space_tracking,Shipment_with_Set_live_SKUs,Temp_type,delivered_in_the_promissed_range_num,programacion_rango_facility_num,entrega_dentro_rango_facility_num,delivered_at_promised_day_num,sm_link
0,49f8a217-ea4e-4433-ad9e-78e625008d8b,9,0,9,9,2,9,22,22.0,e197f049-c172-48f8-a9f9-69e5e520b7a9,47e2d054-8b82-420c-b1b0-18010f57489c,2023-08-19 18:20:41.032,2023-08-18 22:34:35.629,2023-08-19 12:20:41.032,2023-08-19 15:42:07.228,2023-08-19 09:42:07.228,None,None,2023-08-19 15:00:00,2023-08-19 09:00:00,2023-08-19 17:00:00,2023-08-19 11:00:00,7b8a6104-f1e0-4f26-9e5e-b0ce15933ef0,NA-MX-CDMX-Insurgentes Sur,America/Mexico_City,Pluck Botanas,MX,Mexico,2023-08-19,2023-08-01,2023-08-13,,1,0,1,1,1,True,False,False,HAND_DELIVER,None,None,CARLOS GARCIA VILLARREAL,carlosgvcdmx@hotmail.com,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,MEX - Insurgentes Sur,Mexico City,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,2023-08-19 09:42:07.228,9fc7e39b-deb5-4ac3-b569-7ea283cff9b2,None,,DELIVERER_DESIGNATION_OWNER,UNKNOWN,,0.0,0 00:00:00.000,11.0,0 11:07:31.599,158.0,0 02:38:33.804,158.0,0 02:38:33.804,2.642500,0,1,0,0,13.768056,0 days 13:46:05.403000,1,1,1,1,Dry,1,1,1,1,https://stock.cloudkitchens.com/scanner/facili...
1,793e7ce8-92db-4c98-aae6-59bd1637e84f,20,0,20,0,10,20,17,17.0,e3a0f31a-0d4a-4a41-a3a1-e83725e3ce2a,58b818c6-615e-4316-b4a3-b9a6bb914c30,2023-08-12 01:46:57.266,2023-08-11 15:56:15.519,2023-08-11 20:46:57.266,2023-08-11 21:53:04.880,2023-08-11 16:53:04.880,None,None,2023-08-11 22:00:00,2023-08-11 17:00:00,2023-08-11 22:00:00,2023-08-11 17:00:00,0a540d6c-cfbc-48e9-be5a-74bdd7a1a859,SA-CO-BAQ-Alto Prado,America/Bogota,Deli reposteria,CO,Colombia,2023-08-11,2023-08-01,2023-08-06,,1,0,1,1,1,False,True,False,COURIER,,,None,None,None,Courier,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,BAQ - Alto Prado,Barranquilla,SSL,LATAM,None,None,iVBORw0KGgoAAAANSUhEUgAAAZAAAAGQAQAAAACoxAthAA...,SOURCE_WEB_UI,ACCEPTED,None,None,457924e2-c55b-4c58-b9c3-4ac82dfd9a56,2023-08-11 16:53:04.880,457924e2-c55b-4c58-b9c3-4ac82dfd9a56,None,,None,UNKNOWN,,0.0,0 00:00:00.000,0.0,0 00:56:49.361,233.0,0 03:53:52.386,233.0,0 03:53:52.386,3.897778,0,1,0,0,4.844722,0 days 04:50:41.747000,1,0,1,1,Cold,0,1,1,1,https://stock.cloudkitchens.com/scanner/facili...
2,3c0d3ce5-3b2e-4e0b-8254-18ca61f1c2b6,18,0,18,0,9,18,14,15.0,e3a

In [161]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1HlFoCJKRBaM_T46q7BNfbTZLTVBTw2JFpHt9k6I-4NU')

# select sheet

ws = ss.worksheet('title','SM_info')

print('Clearing Range ...')
ws.clear(start='A', end='co')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(por_sm_spread_ssl, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [162]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1BG9l8Rzo-BTKdIPh8D4WaOPa-KWw5hFAecC9mpmjfYw')

# select sheet

ws = ss.worksheet('title','Dataset V1')

print('Clearing Range ...')
ws.clear(start='A', end='co')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(por_sm_spread_2, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [163]:
query_1er_sm="""

with shipment_line_item as (
SELECT
shipment_manifest_uuid,
count(quantity_expected) as num_skus,
sum(case 
when quantity_delivered is null then 1
else 0
end) as non_counted_sku,
sum(case
when quantity_delivered is not null then 1
else 0
end) as counted_sku,
sum(li.quantity_expected) as total_quantity_expected,
sum(li.quantity_delivered) as total_quantity_delivered
FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest_line_item as li
where day_partition>='2020-01-01'


group by 1
)


, first_sm as (

SELECT org_uuid, facility_uuid

,min(created_at) as first_sm_date

from hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest

where day_partition >='01-01-2022'
and deleted_at is null

group by 1,2

)

SELECT 
--uuid,
shipment_line_item.*,

sm.org_uuid,
sm.created_at,
cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ) as created_at_local,


sm.accepted_at,
cast(AT_TIMEZONE(sm.accepted_at, facility.timezone) as TIMESTAMP ) as accepeted_at_local,

delivered_at,
cast(AT_TIMEZONE(sm.delivered_at, facility.timezone) as TIMESTAMP ) as delivered_at_local,

sm.deleted_at as deleted_sm_date,
cast(AT_TIMEZONE(sm.deleted_at, facility.timezone) as TIMESTAMP ) as deleted_at_local,

delivery_window_start,
cast(AT_TIMEZONE(delivery_window_start, facility.timezone) as TIMESTAMP ) as delivery_windows_start_local,

delivery_window_end,
cast(AT_TIMEZONE(delivery_window_end, facility.timezone) as TIMESTAMP ) as delivery_windows_end_local,



transaction_uuid,
facility.name as facility_name,
facility.timezone,
org.name as org_name
,facility.country_code
,dc.country
,date_trunc('day',date(sm.created_at)) as created_day
,date_trunc('month',date(sm.created_at)) as created_month
,date_trunc('week',date(sm.created_at)) as created_week
, sm.comment as comment
, CASE 

  when accepted_at IS NULL THEN 0
  Else 1
  END 

  as accepted
  
,CASE 

  when sm.deleted_at IS NULL THEN 0
  Else 1
  END 

  as deleted


, 

contains_dry_product_type,
contains_cold_product_type,
contains_frozen_product_type,
delivery_type,
company_name,
tracking_id,
deliverer_name,
deliverer_email_address,
other_description,

CASE
WHEN delivery_type = 'COURIER' THEN 'Courier'
WHEN delivery_type = 'HAND_DELIVER' THEN 'Self Drop-Off'
ELSE 'Other'
END AS delivery_method,


CASE
when delivery_window_end < DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 'Overdue Shipment'
when delivery_window_end > DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 'Awaiting Arrival'
when delivery_window_end > accepted_at then 'Shipment Accepted Within Window'
when delivery_window_end < accepted_at then 'Shipment Accepted Outside Window'
END AS acceptance_status,


case 
when total_quantity_delivered <> total_quantity_expected then 'Incorrect Quantity Submitted'
when total_quantity_delivered = total_quantity_expected then 'Correct Quantity Submitted'
when total_quantity_delivered is null and accepted_at is null then 'Not Accepted'
when total_quantity_delivered is null and accepted_at is not null then 'Not Counted'
END AS submission_accuracy


,CASE 
when delivery_window_end < DATE(AT_TIMEZONE(current_timestamp, facility.timezone)) and accepted_at is NULL then 1 else 0
end as is_overdue
,case
when tracking_id is not null and tracking_id <> '' then concat('https://www.google.com/search?q=',tracking_id) 
end as tracking_link


,(
CASE
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when  cast(facility.facility_uuid as VARCHAR ) = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when  cast(facility.facility_uuid as VARCHAR ) = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when  cast(facility.facility_uuid as VARCHAR ) = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   
   else facility.name

  
END
) AS facility_sf_name
  









,concat('https://stock.cloudkitchens.com/scanner/facility/',cast(facility.facility_uuid as VARCHAR ),'/receive-stock/',cast(uuid as varchar)) as sm_link

,cast(AT_TIMEZONE(first_sm_date, facility.timezone) as TIMESTAMP ) as first_sm_local_time



FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest as sm

left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON sm.facility_uuid=cast(facility.facility_uuid as VARCHAR)


left join hudi_ingest.processed.store_config_organization_view as org
on cast(sm.org_uuid as VARCHAR) =org.organization_id

left join shipment_line_item on 
cast(sm.uuid as VARCHAR ) = cast(shipment_line_item.shipment_manifest_uuid as VARCHAR )

left join first_sm
on sm.org_uuid=first_sm.org_uuid
and sm.facility_uuid=first_sm.facility_uuid


JOIN hudi_ingest.scratch.dim_countries dc 
   ON facility.country_code = dc.country_code AND region = 'LATAM'


where sm.day_partition>='01-01-2020'

and sm.created_at=first_sm.first_sm_date

and facility.country_code in ('BR','MX','CO'--,'EC','PE','PR','CL','GT','CR','DO','SV'
)




"""

print("querying ...")
primer_sm = ds_trino.fetch_data(conn=conn, sql_str=query_1er_sm, use_cache=False)
print("pulled")

querying ...
pulled


In [164]:
primer_sm

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,total_quantity_expected,total_quantity_delivered,org_uuid,created_at,created_at_local,accepted_at,accepeted_at_local,delivered_at,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_window_start,delivery_windows_start_local,delivery_window_end,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted,deleted,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,sm_link,first_sm_local_time
0,dfdd2163-f29e-4752-84aa-90d1f6db4ea6,37,0,37,57,57.0,76db5bed-671c-434a-8e9e-fbaa4a083a0b,2022-01-12 10:19:28.111,2022-01-12 07:19:28.111,2022-01-14 15:06:38.491,2022-01-14 12:06:38.491,2022-01-14 15:06:33.143,2022-01-14 12:06:33.143,None,None,2022-01-14 14:30:00.000,2022-01-14 11:30:00.000,2022-01-14 14:30:00.000,2022-01-14 11:30:00.000,0ea0ece4-2e5d-4379-ba76-4fc1b5bd5f03,SA-BR-RJ-Camerino,America/Sao_Paulo,Green Food - CHURN,BR,Brazil,2022-01-12,2022-01-01,2022-01-10,"{""deliveryType"":""hand-deliver"",""productTypes"":...",1,0,False,False,True,HAND_DELIVER,None,None,Patrícia Gil Gomes,patriciagilgomes@hotmail.com,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,RIO - Camerino,https://stock.cloudkitchens.com/scanner/facili...,2022-01-12 07:19:28.111
1,dfbb0b76-2578-4d21-9a1d-eff6d1224c79,23,0,23,330,330.0,57957f56-d5b1-48ce-aec8-abd883bf7d43,2022-02-21 19:58:34.043,2022-02-21 16:58:34.043,2022-03-04 19:41:27.747,2022-03-04 16:41:27.747,2022-03-04 19:41:23.503,2022-03-04 16:41:23.503,None,None,2022-03-04 17:00:00.000,2022-03-04 14:00:00.000,2022-03-04 20:00:00.000,2022-03-04 17:00:00.000,bf4bff3d-5483-4c1d-9f0e-b01a554b0fb3,SA-BR-BH-Rua Paracatu,America/Sao_Paulo,Prússia Bier - Churn,BR,Brazil,2022-02-21,2022-02-01,2022-02-21,,1,0,False,True,False,COURIER,,,None,None,None,Courier,Shipment Accepted Within Window,Correct Quantity Submitted,0,None,BHZ - Paracatu,https://stock.cloudkitchens.com/scanner/facili...,2022-02-21 16:58:34.043
2,7182ff51-29a1-4c41-8038-5406359eab34,5,0,5,0,0.0,1976e5b7-83e3-4a24-93ec-b03ce6d7ee16,2021-08-11 19:17:15.672,2021-08-11 14:17:15.672,2021-08-18 00:55:47.555,2021-08-17 19:55:47.555,2021-08-18 00:55:47.155,2021-08-17 19:55:47.155,None,None,2021-08-12 15:00:00.000,2021-08-12 10:00:00.000,2021-08-12 17:00:00.000,2021-08-12 12:00:00.000,296635e0-11ad-466e-abef-901f4d24c144,NA-MX-CDMX-Santa Fe,America/Mexico_City,Tamalli,MX,Mexico,2021-08-11,2021-08-01,2021-08-09,"{""deliveryType"":""hand-deliver"",""productTypes"":...",1,0,False,False,True,HAND_DELIVER,None,None,Philippe Robert,,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,MEX - Santa Fe,https://stock.cloudkitchens.com/scanner/facili...,2021-08-11 14:17:15.672
3,4b4e3d64-e564-40f2-b7ce-3aa93d1f6d29,20,0,20,93,93.0,45015f48-ae8a-4813-99ae-8b7e8c1dedc5,2021-10-25 19:49:08.405,2021-10-25 16:49:08.405,2021-10-26 17:11:18.798,2021-10-26 14:11:18.798,2021-10-26 17:10:36.450,2021-10-26 14:10:36.450,None,None,2021-10-26 12:00:00.000,2021-10-26 09:00:00.000,2021-10-26 12:30:00.000,2021-10-26 09:30:00.000,6077bbee-4422-4132-adc3-1fcaa7583dc0,SA-BR-RJ-Uptown,America/Sao_Paulo,Marmifit - CHURN,BR,Brazil,2021-10-25,2021-10-01,2021-10-25,"{""productTypes"":[""frozen""],""deliveryType"":""han...",1,0,False,False,True,HAND_DELIVER,None,None,Wesley Luiz,,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,RIO - Barra,https://stock.cloudkitchens.com/scanner/facili...,2021-10-25 16:49:08.405
4,c720fcda-25c4-4438-8222-9a841d811e86,159,4,155,657,653.0,f640cfc9-e1e8-4ad1-b2f9-8d640b5e9d69,2022-07-25 20:06:34.139,2022-07-25 15:06:34.139,2022-07-29 17:49:49.366,2022-07-29 12:49:49.366,2022-07-29 17

In [165]:
primer_sm.columns

Index(['shipment_manifest_uuid', 'num_skus', 'non_counted_sku', 'counted_sku',
       'total_quantity_expected', 'total_quantity_delivered', 'org_uuid',
       'created_at', 'created_at_local', 'accepted_at', 'accepeted_at_local',
       'delivered_at', 'delivered_at_local', 'deleted_sm_date',
       'deleted_at_local', 'delivery_window_start',
       'delivery_windows_start_local', 'delivery_window_end',
       'delivery_windows_end_local', 'transaction_uuid', 'facility_name',
       'timezone', 'org_name', 'country_code', 'country', 'created_day',
       'created_month', 'created_week', 'comment', 'accepted', 'deleted',
       'contains_dry_product_type', 'contains_cold_product_type',
       'contains_frozen_product_type', 'delivery_type', 'company_name',
       'tracking_id', 'deliverer_name', 'deliverer_email_address',
       'other_description', 'delivery_method', 'acceptance_status',
       'submission_accuracy', 'is_overdue', 'tracking_link',
       'facility_sf_name', 'sm_link'

In [166]:
primer_sm_spread=primer_sm[['shipment_manifest_uuid', 'num_skus', 'non_counted_sku', 'counted_sku',
       'total_quantity_expected', 'total_quantity_delivered', 'org_uuid',
        'created_at_local', 'accepeted_at_local',
        'delivered_at_local', 'deleted_sm_date',
       'deleted_at_local', 
       'delivery_windows_start_local', 
       'delivery_windows_end_local', 'transaction_uuid', 'facility_name',
       'timezone', 'org_name', 'country_code', 'country', 'created_day',
       'created_month', 'created_week', 'comment', 'accepted', 'deleted',
       'contains_dry_product_type', 'contains_cold_product_type',
       'contains_frozen_product_type', 'delivery_type', 'company_name',
       'tracking_id', 'deliverer_name', 'deliverer_email_address',
       'other_description', 'delivery_method', 'acceptance_status',
       'submission_accuracy', 'is_overdue', 'tracking_link',
       'facility_sf_name', 'sm_link', 'first_sm_local_time']]

In [167]:
primer_sm_spread

,shipment_manifest_uuid,num_skus,non_counted_sku,counted_sku,total_quantity_expected,total_quantity_delivered,org_uuid,created_at_local,accepeted_at_local,delivered_at_local,deleted_sm_date,deleted_at_local,delivery_windows_start_local,delivery_windows_end_local,transaction_uuid,facility_name,timezone,org_name,country_code,country,created_day,created_month,created_week,comment,accepted,deleted,contains_dry_product_type,contains_cold_product_type,contains_frozen_product_type,delivery_type,company_name,tracking_id,deliverer_name,deliverer_email_address,other_description,delivery_method,acceptance_status,submission_accuracy,is_overdue,tracking_link,facility_sf_name,sm_link,first_sm_local_time
0,dfdd2163-f29e-4752-84aa-90d1f6db4ea6,37,0,37,57,57.0,76db5bed-671c-434a-8e9e-fbaa4a083a0b,2022-01-12 07:19:28.111,2022-01-14 12:06:38.491,2022-01-14 12:06:33.143,None,None,2022-01-14 11:30:00.000,2022-01-14 11:30:00.000,0ea0ece4-2e5d-4379-ba76-4fc1b5bd5f03,SA-BR-RJ-Camerino,America/Sao_Paulo,Green Food - CHURN,BR,Brazil,2022-01-12,2022-01-01,2022-01-10,"{""deliveryType"":""hand-deliver"",""productTypes"":...",1,0,False,False,True,HAND_DELIVER,None,None,Patrícia Gil Gomes,patriciagilgomes@hotmail.com,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,RIO - Camerino,https://stock.cloudkitchens.com/scanner/facili...,2022-01-12 07:19:28.111
1,dfbb0b76-2578-4d21-9a1d-eff6d1224c79,23,0,23,330,330.0,57957f56-d5b1-48ce-aec8-abd883bf7d43,2022-02-21 16:58:34.043,2022-03-04 16:41:27.747,2022-03-04 16:41:23.503,None,None,2022-03-04 14:00:00.000,2022-03-04 17:00:00.000,bf4bff3d-5483-4c1d-9f0e-b01a554b0fb3,SA-BR-BH-Rua Paracatu,America/Sao_Paulo,Prússia Bier - Churn,BR,Brazil,2022-02-21,2022-02-01,2022-02-21,,1,0,False,True,False,COURIER,,,None,None,None,Courier,Shipment Accepted Within Window,Correct Quantity Submitted,0,None,BHZ - Paracatu,https://stock.cloudkitchens.com/scanner/facili...,2022-02-21 16:58:34.043
2,7182ff51-29a1-4c41-8038-5406359eab34,5,0,5,0,0.0,1976e5b7-83e3-4a24-93ec-b03ce6d7ee16,2021-08-11 14:17:15.672,2021-08-17 19:55:47.555,2021-08-17 19:55:47.155,None,None,2021-08-12 10:00:00.000,2021-08-12 12:00:00.000,296635e0-11ad-466e-abef-901f4d24c144,NA-MX-CDMX-Santa Fe,America/Mexico_City,Tamalli,MX,Mexico,2021-08-11,2021-08-01,2021-08-09,"{""deliveryType"":""hand-deliver"",""productTypes"":...",1,0,False,False,True,HAND_DELIVER,None,None,Philippe Robert,,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,MEX - Santa Fe,https://stock.cloudkitchens.com/scanner/facili...,2021-08-11 14:17:15.672
3,4b4e3d64-e564-40f2-b7ce-3aa93d1f6d29,20,0,20,93,93.0,45015f48-ae8a-4813-99ae-8b7e8c1dedc5,2021-10-25 16:49:08.405,2021-10-26 14:11:18.798,2021-10-26 14:10:36.450,None,None,2021-10-26 09:00:00.000,2021-10-26 09:30:00.000,6077bbee-4422-4132-adc3-1fcaa7583dc0,SA-BR-RJ-Uptown,America/Sao_Paulo,Marmifit - CHURN,BR,Brazil,2021-10-25,2021-10-01,2021-10-25,"{""productTypes"":[""frozen""],""deliveryType"":""han...",1,0,False,False,True,HAND_DELIVER,None,None,Wesley Luiz,,None,Self Drop-Off,Shipment Accepted Outside Window,Correct Quantity Submitted,0,None,RIO - Barra,https://stock.cloudkitchens.com/scanner/facili...,2021-10-25 16:49:08.405
4,c720fcda-25c4-4438-8222-9a841d811e86,159,4,155,657,653.0,f640cfc9-e1e8-4ad1-b2f9-8d640b5e9d69,2022-07-25 15:06:34.139,2022-07-29 12:49:49.366,2022-07-29 12:49:28.112,None,None,2022-07-27 09:00:00.000,2022-07-27 17:00:00.000,5f5d1edd-5e03-458b-a19a-5635c71818be,NA-MX-CDMX-Lindavista,America/Mexico_City,Aguillon Beauty Supply,MX,Mexico,2022-07-25,2022-07-01,2022-07-25,,1,0,True,False,False,COURIER,Travisa,161298MT504811,None,None,None,Courier,Shipment Accepted Outside Window,Incorrect Quantity Submitted,0,https://www.google.com/search?q=161298MT504811,MEX - Lindavista,https://stock.cloudkitchens.com/scanner/facili...,2022-07-25 15:06:34.139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [168]:
# Everything ready to upload to GSheets

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1k1mEBO28vtb56z645ZzPPdPKQa-DrYbzV6dguoFO-gA')

# select sheet

ws = ss.worksheet('title','data')

print('Clearing Range ...')
ws.clear(start='A', end='AI')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(primer_sm_spread, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [169]:
import dateutil.relativedelta

In [170]:
hoy=datetime.datetime.today()
hoy

datetime.datetime(2023, 9, 13, 20, 21, 52, 857413)

In [171]:
por_sm['calculation_time']=hoy

# big query 

In [174]:
from ds.utilities.secrets import secrets
from google.oauth2 import service_account
from google.cloud import bigquery
s = secrets.Secrets()
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"))
client = bigquery.Client(credentials=credentials)

In [175]:
por_sku_3.to_gbq(credentials = credentials ,
              destination_table='latam_cr_growth.shipment_manifest_per_sku',
              progress_bar = True,
              if_exists='replace'
              #if_exists='append'
                        )

1it [00:42, 42.51s/it]


In [176]:
query_agrupado_por_sm_sku="""

with sm_sku_line as (


SELECT 
sm.uuid as sm_uuid,
sm_sku.line_item_uuid as line_item_id,
--facility.facility_uuid as facility_id,
org_uuid,
org.name as org_name,
sm.created_at,
cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ) as created_at_local,

created_by_user_uuid,
sm.accepted_at,
cast(AT_TIMEZONE(sm.accepted_at, facility.timezone) as TIMESTAMP ) as accepeted_at_local,

sm.delivered_at,
cast(AT_TIMEZONE(sm.delivered_at, facility.timezone) as TIMESTAMP ) as delivered_at_local,

delivery_window_start,
cast(AT_TIMEZONE(delivery_window_start, facility.timezone) as TIMESTAMP ) as delivery_windows_start_local,

delivery_window_end,
cast(AT_TIMEZONE(delivery_window_end, facility.timezone) as TIMESTAMP ) as delivery_windows_end_local,

transaction_uuid,
sm.facility_uuid as facility_id,
facility.name as facility_name,
facility.country_code as country_code
,dc.country
,(CASE 

  when dc.country='Brazil' then 'Brazil'
  when dc.country = 'United States' then 'US/CAN'
  else 'SSL'
  end) as subregion

,(case when facility.country_code in ('MX','CO','BR') then 'LATAM'
        when facility.country_code in ('US') then 'US/CAN'
         end ) as region



,date_trunc('day',date(sm.created_at)) as created_day
,date_trunc('month',date(sm.created_at)) as created_month
--,date_trunc('week',date(sm.created_at)) as created_week

,date_trunc('week', DATE(cast(AT_TIMEZONE(sm.created_at, facility.timezone) as TIMESTAMP ))+ interval '1' day)- interval '1' day as created_week


,sm_sku.sku_uuid
,sm_sku.quantity_expected
,sm_sku.quantity_delivered
,sku.name as sku_name
,sku.manufacturer_brand_name as brand_name
,sm.comment
, CASE 

  when sm.accepted_at IS NULL THEN 0
  Else 1
  END 

  as accepted
  
,CASE 

  when sm.deleted_at IS NULL THEN 0
  Else 1
  END 

  as deleted
  
,(
CASE
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN cast(facility.facility_uuid as VARCHAR ) = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN cast(facility.facility_uuid as VARCHAR ) = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when  cast(facility.facility_uuid as VARCHAR ) = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when  cast(facility.facility_uuid as VARCHAR ) = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when  cast(facility.facility_uuid as VARCHAR ) = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN  cast(facility.facility_uuid as VARCHAR ) = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
  
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN  cast(facility.facility_uuid as VARCHAR ) = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   
   else facility.name

  
END
) AS facility_sf_name

,(CASE
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
       cast(facility.facility_uuid as VARCHAR ) in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
       cast(facility.facility_uuid as VARCHAR ) IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        cast(facility.facility_uuid as VARCHAR ) IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
         cast(facility.facility_uuid as VARCHAR ) IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
        
        else 'other city'
END
) AS city




  
,space_uuid
,	expiration_date
,cast(AT_TIMEZONE(expiration_date, facility.timezone) as TIMESTAMP ) as expiration_date_local

,	live_at
,cast(AT_TIMEZONE(live_at, facility.timezone) as TIMESTAMP ) as live_at_local

,sm.delivery_sort
,cast(AT_TIMEZONE(sm.delivery_sort, facility.timezone) as TIMESTAMP ) as delivery_sort_local
,delivery_type, company_name, other_description

,shipment_receiving.vendor_expiration, facility_expiration, shipment_receiving.receiving_to_accepted_seconds, shipment_receiving.delivered_to_receiving_seconds
, status

FROM hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest as sm

left JOIN hudi_ingest.fos_restaurant_platform_cockroach_dbreplica.facility AS facility 
ON sm.facility_uuid=cast(facility.facility_uuid as VARCHAR )

left join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.shipment_manifest_line_item as sm_sku
on sm.uuid=cast(sm_sku.shipment_manifest_uuid as VARCHAR )

left JOIN hudi_ingest.skusku_restaurant_platform_cockroach_dbreplica.sku AS sku
on sm_sku.sku_uuid=cast(sku.sku_id as VARCHAR )

left join hudi_ingest.processed.store_config_organization_view as org
on sm.org_uuid=cast(org.organization_id as VARCHAR)

left join hudi_ingest.processed.inventory_shipment_receiving as shipment_receiving
on sm_sku.sku_uuid = shipment_receiving.sku_uuid
and cast(sm_sku.shipment_manifest_uuid as VARCHAR ) = shipment_receiving.shipment_manifest_uuid



JOIN hudi_ingest.scratch.dim_countries dc 
   ON facility.country_code = dc.country_code 
   --AND region = 'LATAM'


where facility.country_code in ('BR','MX','CO'--'US'--,'EC','PE','PR','CL','GT','CR','DO','SV'
)
and date_trunc('day',date(sm.day_partition)) >= DATE_ADD('Day' ,-180, current_date)
and sm.day_partition>='01-01-2021'
and sm_sku.day_partition>='01-01-2021'
and sku.day_partition>='01-01-2021'
--and sm.uuid in ('e153010c-fd1c-41c9-aa84-44a2590bbc85', 'ce848b57-0fa0-470c-b777-df2a3207d01f')



)


select sm_uuid, country_code, country, region, subregion, created_day, 
created_month, created_week, brand_name,


org_uuid, 
org_name, facility_id, facility_name, facility_sf_name,
sku_uuid, sku_name, 
status, created_at_local, accepeted_at_local, delivery_windows_start_local, delivery_windows_end_local, transaction_uuid

,sum(quantity_expected) as sum_expected
,sum(quantity_delivered) as sum_delivered, count(DISTINCT (space_uuid)) as number_unique_space_uuid
,count(DISTINCT (line_item_id)) as num_item_id
, count(space_uuid) as num_space_uuid
, count(expiration_date) as num_exp_date

, sum (case when expiration_date is not null and (quantity_expected is not null or quantity_expected<>0 )then 1 else 0 end) as customer_type_exp_date
, sum (case when expiration_date is not null and (quantity_expected is null or quantity_expected=0)then 1 else 0 end) as facility_type_exp_date

, case when count(space_uuid)>0 then 1 else 0 end as has_space_uuid
, case when count(expiration_date)>0 then 1 else 0 end as has_exp_date

from sm_sku_line

group by 1,2,3,4,5,6,7, 8,9,10,11,12,13,14,15,16,17,18,19,20,21,22


"""

print("querying ...")
sm_sku_agrupado = ds_trino.fetch_data(conn=conn, sql_str=query_agrupado_por_sm_sku, use_cache=False)
print("pulled")


querying ...
pulled


In [177]:
sm_sku_agrupado

,sm_uuid,country_code,country,region,subregion,created_day,created_month,created_week,brand_name,org_uuid,org_name,facility_id,facility_name,facility_sf_name,sku_uuid,sku_name,status,created_at_local,accepeted_at_local,delivery_windows_start_local,delivery_windows_end_local,transaction_uuid,sum_expected,sum_delivered,number_unique_space_uuid,num_item_id,num_space_uuid,num_exp_date,customer_type_exp_date,facility_type_exp_date,has_space_uuid,has_exp_date
0,dbd86ebb-e474-4c89-9ce0-963eec672af4,BR,Brazil,LATAM,Brazil,2023-04-25,2023-04-01,2023-04-23,Cervejaria Pontal,e30fb1b1-4b5c-45b4-88de-6af298b132ed,Cervejaria Pontal - Pik n' Pak,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,SA-BR-RJ-Tijuca,RIO - Tijuca,dc343d11-a2d8-42ad-9ac5-378e287e4ac1,Chope Pontal Friburgo Lager 1 Litro | Cervejar...,ACCEPTED,2023-04-25 19:28:34.420,2023-04-26 15:13:21.426,2023-04-26 11:00:00.000,2023-04-26 15:00:00.000,b0f57a0c-2b9f-48ea-b265-23c52a456289,4,4.0,1,2,1,2,2,1,1,1
1,cd5a352f-d2ec-4bb7-a843-ab5ff57a58ef,MX,Mexico,LATAM,SSL,2023-06-13,2023-06-01,2023-06-11,Jaso Bakery,6512e2d1-a0fd-4da1-8dff-76141a13f562,Jaso Bakery,9101d4f9-828f-45c5-b8b6-16986e76b374,NA-MX-CDMX-Pedregal*,MEX - Pedregal,7060cb9b-3fbb-48c5-bddc-a6647b16b410,Rebanada Jaime Kmil,ACCEPTED,2023-06-13 10:50:22.931,2023-06-13 11:31:32.885,2023-06-13 08:30:00.000,2023-06-13 11:00:00.000,45ec71cf-4655-4623-97af-86f25f66b62e,2,2.0,1,2,1,2,2,1,1,1
2,705285f5-ac21-41a9-b269-cd176bfc0766,BR,Brazil,LATAM,Brazil,2023-04-17,2023-04-01,2023-04-16,Hilla Doces,0eb6a959-e305-4bce-a678-27e910729eb6,Hilla Doces - Pik n' Pak,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,SAO - Dino,37bbd246-2cf8-4bbc-846e-e0804860b336,Bolo caseirinho cenoura com brigadeiro - 300g ...,ACCEPTED,2023-04-17 11:31:27.079,2023-04-18 17:58:56.031,2023-04-18 09:00:00.000,2023-04-18 17:00:00.000,a8322758-ee95-4ddd-a7ee-9d6c8ad665bd,6,6.0,1,2,1,2,2,1,1,1
3,f9b36f25-e494-4f32-b438-79411de9a3cb,MX,Mexico,LATAM,SSL,2023-03-31,2023-03-01,2023-03-26,Flor de Lis,4032b879-e778-4a03-adb6-527f7da04494,Flor de Lis CR,da27f06f-01fd-4106-88d9-b3fd41446515,NA-MX-CDMX-Lindavista,MEX - Lindavista,4fddc4ce-c842-4041-b1ee-a9bb4716f3ea,1 TAMAL RAJAS CON QUESO,ACCEPTED,2023-03-31 12:10:19.126,2023-04-05 11:22:50.159,2023-04-05 09:00:00.000,2023-04-05 17:00:00.000,8482ce79-93ab-4971-872c-b60a2bfd1d28,8,8.0,1,2,1,0,0,0,1,0
4,cf0851ec-3d11-4480-b836-efb15761e461,BR,Brazil,LATAM,Brazil,2023-06-03,2023-06-01,2023-05-28,Vó Luiza Bolos - Pik n' Pak,2358606e-8992-4700-9bef-8ab390f3d152,Vó Luiza Bolos - Pik n' Pak,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,SAO - Dino,9b89937c-b30c-46ec-85e7-905e790bdd6c,Bolo de Brigadeiro de Paçoca | Vó Luiza Bolos,ACCEPTED,2023-06-03 16:44:15.123,2023-06-03 17:56:01.312,2023-06-03 16:15:00.000,2023-06-03 17:00:00.000,6779be34-e3da-4c6c-b5f3-ead1ca7b8455,3,3.0,1,2,1,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174883,3b1b7064-90bf-415e-9502-f3cd2729f90b,BR,Brazil,LATAM,Brazil,2023-06-27,2023-06-01,2023-06-25,Gelati,ffe35195-a276-4537-8072-6875e100e579,Gelati - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,SA-BR-RJ-Botafogo,RIO - Botafogo,3b556fdc-ea79-4c79-a534-2e10a18c7ce8,Milho Verde 500ml - Gelati,ACCEPTED,2023-06-27 12:34:49.019,2023-06-27 13:32:14.967,2023-06-27 09:00:00.000,2023-06-27 17:00:00.000,5c1edc6e-af9a-4b94-b395-c1a4272b2a9b,10,10.0,1,2,1,2,2,1,1,1
174884,5fbda239-f199-4d31-bfbc-242902e158a4,BR,Brazil,LATAM,Brazil,2023-05-18,2023-05-01,2023-05-14,Gelati,ffe35195-a276-4537-8072-6875e100e579,Gelati - Pik n' Pak,0ee5dccf-ed5b-465c-9858-b53351353e3b,SA-BR-RJ-Camerino,RIO - Camerino,7dc81f73-5520-4594-8bdf-3d80d32d7a90,Paçoca 500ml - Gelati,ACCEPTED,2023-05-18 12:51:13.063,2023-05-18 16:51:32.880,2023-05-18 09:00:00.000,2023-05-18 17:00:00.000,cc7ecae9-42fd-4aef-b764-90b5a062e38e,9,9.0,1,2,1,0,0,0,1,0
174885,5fbda239-f199-4d31-bfbc-242902e158a4,BR,Brazil,LATAM,Brazil,2023-05-18,2023-05-01,202

In [178]:
sm_sku_agrupado.columns

Index(['sm_uuid', 'country_code', 'country', 'region', 'subregion',
       'created_day', 'created_month', 'created_week', 'brand_name',
       'org_uuid', 'org_name', 'facility_id', 'facility_name',
       'facility_sf_name', 'sku_uuid', 'sku_name', 'status',
       'created_at_local', 'accepeted_at_local',
       'delivery_windows_start_local', 'delivery_windows_end_local',
       'transaction_uuid', 'sum_expected', 'sum_delivered',
       'number_unique_space_uuid', 'num_item_id', 'num_space_uuid',
       'num_exp_date', 'customer_type_exp_date', 'facility_type_exp_date',
       'has_space_uuid', 'has_exp_date'],
      dtype='object')

In [179]:
sm_sku_agrupado['created_at_local']=pd.to_datetime(sm_sku_agrupado['created_at_local'], utc=False, errors='coerce')

In [180]:
sm_sku_agrupado['accepeted_at_local']=pd.to_datetime(sm_sku_agrupado['accepeted_at_local'], utc=False, errors='coerce')

In [181]:
sm_sku_agrupado['created_day']=pd.to_datetime(sm_sku_agrupado['created_day'], utc=False, errors='coerce')

In [182]:
sm_sku_agrupado['created_month']=pd.to_datetime(sm_sku_agrupado['created_month'], utc=False, errors='coerce')

In [183]:
sm_sku_agrupado['created_week']=pd.to_datetime(sm_sku_agrupado['created_week'], utc=False, errors='coerce')

In [184]:
#sm_sku_agrupado['delivered_at_local']=pd.to_datetime(sm_sku_agrupado['delivered_at_local'], utc=False, errors='coerce')

In [185]:
sm_sku_agrupado['delivery_windows_start_local']=pd.to_datetime(sm_sku_agrupado['delivery_windows_start_local'], utc=False, errors='coerce')

In [186]:
sm_sku_agrupado['delivery_windows_end_local']=pd.to_datetime(sm_sku_agrupado['delivery_windows_end_local'], utc=False, errors='coerce')

In [187]:
sm_sku_agrupado

,sm_uuid,country_code,country,region,subregion,created_day,created_month,created_week,brand_name,org_uuid,org_name,facility_id,facility_name,facility_sf_name,sku_uuid,sku_name,status,created_at_local,accepeted_at_local,delivery_windows_start_local,delivery_windows_end_local,transaction_uuid,sum_expected,sum_delivered,number_unique_space_uuid,num_item_id,num_space_uuid,num_exp_date,customer_type_exp_date,facility_type_exp_date,has_space_uuid,has_exp_date
0,dbd86ebb-e474-4c89-9ce0-963eec672af4,BR,Brazil,LATAM,Brazil,2023-04-25,2023-04-01,2023-04-23,Cervejaria Pontal,e30fb1b1-4b5c-45b4-88de-6af298b132ed,Cervejaria Pontal - Pik n' Pak,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,SA-BR-RJ-Tijuca,RIO - Tijuca,dc343d11-a2d8-42ad-9ac5-378e287e4ac1,Chope Pontal Friburgo Lager 1 Litro | Cervejar...,ACCEPTED,2023-04-25 19:28:34.420,2023-04-26 15:13:21.426,2023-04-26 11:00:00,2023-04-26 15:00:00,b0f57a0c-2b9f-48ea-b265-23c52a456289,4,4.0,1,2,1,2,2,1,1,1
1,cd5a352f-d2ec-4bb7-a843-ab5ff57a58ef,MX,Mexico,LATAM,SSL,2023-06-13,2023-06-01,2023-06-11,Jaso Bakery,6512e2d1-a0fd-4da1-8dff-76141a13f562,Jaso Bakery,9101d4f9-828f-45c5-b8b6-16986e76b374,NA-MX-CDMX-Pedregal*,MEX - Pedregal,7060cb9b-3fbb-48c5-bddc-a6647b16b410,Rebanada Jaime Kmil,ACCEPTED,2023-06-13 10:50:22.931,2023-06-13 11:31:32.885,2023-06-13 08:30:00,2023-06-13 11:00:00,45ec71cf-4655-4623-97af-86f25f66b62e,2,2.0,1,2,1,2,2,1,1,1
2,705285f5-ac21-41a9-b269-cd176bfc0766,BR,Brazil,LATAM,Brazil,2023-04-17,2023-04-01,2023-04-16,Hilla Doces,0eb6a959-e305-4bce-a678-27e910729eb6,Hilla Doces - Pik n' Pak,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,SAO - Dino,37bbd246-2cf8-4bbc-846e-e0804860b336,Bolo caseirinho cenoura com brigadeiro - 300g ...,ACCEPTED,2023-04-17 11:31:27.079,2023-04-18 17:58:56.031,2023-04-18 09:00:00,2023-04-18 17:00:00,a8322758-ee95-4ddd-a7ee-9d6c8ad665bd,6,6.0,1,2,1,2,2,1,1,1
3,f9b36f25-e494-4f32-b438-79411de9a3cb,MX,Mexico,LATAM,SSL,2023-03-31,2023-03-01,2023-03-26,Flor de Lis,4032b879-e778-4a03-adb6-527f7da04494,Flor de Lis CR,da27f06f-01fd-4106-88d9-b3fd41446515,NA-MX-CDMX-Lindavista,MEX - Lindavista,4fddc4ce-c842-4041-b1ee-a9bb4716f3ea,1 TAMAL RAJAS CON QUESO,ACCEPTED,2023-03-31 12:10:19.126,2023-04-05 11:22:50.159,2023-04-05 09:00:00,2023-04-05 17:00:00,8482ce79-93ab-4971-872c-b60a2bfd1d28,8,8.0,1,2,1,0,0,0,1,0
4,cf0851ec-3d11-4480-b836-efb15761e461,BR,Brazil,LATAM,Brazil,2023-06-03,2023-06-01,2023-05-28,Vó Luiza Bolos - Pik n' Pak,2358606e-8992-4700-9bef-8ab390f3d152,Vó Luiza Bolos - Pik n' Pak,87a40094-cf48-4032-8166-e4cee61bce2f,BR - Sao Paulo - Dino,SAO - Dino,9b89937c-b30c-46ec-85e7-905e790bdd6c,Bolo de Brigadeiro de Paçoca | Vó Luiza Bolos,ACCEPTED,2023-06-03 16:44:15.123,2023-06-03 17:56:01.312,2023-06-03 16:15:00,2023-06-03 17:00:00,6779be34-e3da-4c6c-b5f3-ead1ca7b8455,3,3.0,1,2,1,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174883,3b1b7064-90bf-415e-9502-f3cd2729f90b,BR,Brazil,LATAM,Brazil,2023-06-27,2023-06-01,2023-06-25,Gelati,ffe35195-a276-4537-8072-6875e100e579,Gelati - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,SA-BR-RJ-Botafogo,RIO - Botafogo,3b556fdc-ea79-4c79-a534-2e10a18c7ce8,Milho Verde 500ml - Gelati,ACCEPTED,2023-06-27 12:34:49.019,2023-06-27 13:32:14.967,2023-06-27 09:00:00,2023-06-27 17:00:00,5c1edc6e-af9a-4b94-b395-c1a4272b2a9b,10,10.0,1,2,1,2,2,1,1,1
174884,5fbda239-f199-4d31-bfbc-242902e158a4,BR,Brazil,LATAM,Brazil,2023-05-18,2023-05-01,2023-05-14,Gelati,ffe35195-a276-4537-8072-6875e100e579,Gelati - Pik n' Pak,0ee5dccf-ed5b-465c-9858-b53351353e3b,SA-BR-RJ-Camerino,RIO - Camerino,7dc81f73-5520-4594-8bdf-3d80d32d7a90,Paçoca 500ml - Gelati,ACCEPTED,2023-05-18 12:51:13.063,2023-05-18 16:51:32.880,2023-05-18 09:00:00,2023-05-18 17:00:00,cc7ecae9-42fd-4aef-b764-90b5a062e38e,9,9.0,1,2,1,0,0,0,1,0
174885,5fbda239-f199-4d31-bfbc-242902e158a4,BR,Brazil,LATAM,Brazil,2023-05-18,2023-05-01,2023-05-14,Gelati,ffe35195-a276-4537-8072-6875e100e579,Gela

In [188]:
sm_sku_agrupado.to_gbq(credentials = credentials ,
              destination_table='latam_cr_growth.shipment_manifest_grouped_per_sku',
              progress_bar = True,
              if_exists='replace'
              #if_exists='append'
                        )

1it [00:24, 24.54s/it]


In [189]:
#clientes activos por facility 1 meses previos
query_por_facility = """

WITH first_order as (
SELECT 
store_id, 
MIN(reference_time/1000) as first_order
FROM hudi_ingest.analytics_views.customer_orders co
WHERE day_partition >= '2020-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2020-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53'  -- SAO - Vargas
                )
GROUP BY 1
            )
            
            



SELECT 

'LATAM' as region,
dc.country,
(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82'  -- BOG - Cedritos
      )
   THEN
      'Bogota'
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4'  -- GDL - Ciudad del Sol
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b'  -- RIO - Camerino
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '88aaaf99-a444-4e0f-99f6-267b635faa53'  -- SAO - Vargas
      )
   then
      'Sao Paulo'
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
END
) AS facility_name_n,
organization_name,
facility_country_code,
facility_name,
organization_id

--ofo_slug,
--date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) < date_add('week', 4, date_trunc('week', DATE(from_unixtime(first_order, facility_timezone)))) as is_new_store,
--is_cancelled,
--subtotal,
--total,




FROM hudi_ingest.analytics_views.customer_orders co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
LEFT JOIN first_order fo 
   ON co.store_id=fo.store_id

 --4 meses   
WHERE date_trunc('day',date(day_partition)) >= DATE_ADD('Day' ,-31, current_date)
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND NOT is_cancelled
AND tenant_type in ('VIRTUAL','CLOUD_RETAIL')
-- AND organization_id = '2a694864-3f0c-45e6-8a8d-d6088e0d122f'
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            
-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53'  -- SAO - Vargas
                )
                
                GROUP by 1,2,3,4,5,6,7,8
 


"""

print("querying ...")
#por_facility = ds_trino.fetch_data(conn=conn, sql_str=query_por_facility, use_cache=False)
print("pulled")

querying ...
pulled


In [190]:
#por_facility

In [191]:
import dateutil.relativedelta

In [192]:
#hoy=datetime.datetime.today()
#hoy

In [193]:
mes=hoy.replace(day=1,hour=0, minute=0, second=0, microsecond=0)

In [194]:
mes

datetime.datetime(2023, 9, 1, 0, 0)

In [195]:
mes_pasado=mes- dateutil.relativedelta.relativedelta(months=1)

In [196]:
mes_antepasado=mes- dateutil.relativedelta.relativedelta(months=2)

In [197]:
#por_facility['facility_name'][5] in por_sm['facility_name'].values

In [198]:
#mes_pasado==por_sm['delivery_window_start'][0].replace(day=1,hour=0, minute=0, second=0, microsecond=0)

In [199]:
por_sm['created_month']=pd.to_datetime(por_sm['created_month'], errors='coerce')

In [200]:
#por_sm_mes_actual=por_sm[por_sm['created_month']==mes]

In [201]:
#por_sm_mes_pasado=por_sm[por_sm['created_month']==mes_pasado]

In [202]:
#por_sm_mes_antepasado=por_sm[por_sm['created_month']==mes_antepasado]

In [203]:
#creo_mes_actual=[]
#creo_mes_pasado=[]
#creo_mes_antepasado=[]
#for i in range(len(por_facility)):
    #valor_facility_1=por_facility['facility_name'][i] in por_sm_mes_actual['facility_name'].values
    #valor_org_1=por_facility['organization_name'][i] in por_sm_mes_actual['org_name'].values
    
    #valor_facility_2=por_facility['facility_name'][i] in por_sm_mes_pasado['facility_name'].values
    #valor_org_2=por_facility['organization_name'][i] in por_sm_mes_pasado['org_name'].values
    
    #valor_facility_3=por_facility['facility_name'][i] in por_sm_mes_antepasado['facility_name'].values
    #valor_org_3=por_facility['organization_name'][i] in por_sm_mes_antepasado['org_name'].values
    #print(valor_facility_1,valor_org_1)
    
    #if valor_facility_1== True and valor_org_1== True:
        #creo_mes_actual.append(True)
    #else:
        #creo_mes_actual.append(False)
    
    #if valor_facility_2== True and valor_org_2== True:
        #creo_mes_pasado.append(True)
    #else:
        #creo_mes_pasado.append(False)
    
    #if valor_facility_3== True and valor_org_3== True:
        #creo_mes_antepasado.append(True)
    #else:
        #creo_mes_antepasado.append(False)
    

In [204]:
#por_facility['created_sm_current_month']=creo_mes_actual
#por_facility['created_sm_last_month']=creo_mes_pasado
#por_facility['created_sm_last_two_month']=creo_mes_antepasado

In [205]:
#mando a big query
por_sm.to_gbq(credentials = credentials ,
              destination_table='latam_cr_growth.shipment_manifest_per_sm',
              progress_bar = True,
              if_exists='replace'
              #if_exists='append'
                        )

1it [00:21, 21.34s/it]


In [206]:
## GSheets Oauth
print('Init Auth')
#gc = pygsheets.authorize(custom_credentials=my_credentials)


# open the spreedsheet
print('Opening Spreadsheet')

#ss = gc.open_by_key('1Ffh0lxSI31wEckOrbpB1gxgd4ry3GdXO62XvksepGds')

# select sheet

#ws = ss.worksheet('title','clientes_activos_facility')

print('Clearing Range ...')
#ws.clear(start='A', end='x')
print('Range CLEARED')

print('Pasting Info ...')
#ws.set_dataframe(por_facility, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [207]:
#clientes activos por facility 6 meses previos
query_por_cliente = """


WITH first_order as (
SELECT 
store_id, 
MIN(reference_time/1000) as first_order
FROM hudi_ingest.analytics_views.customer_orders co
WHERE day_partition >= '2020-01-01'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2020-01-01'
AND NOT is_cancelled
AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
            'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            '6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            'd65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            '9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
            '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol

-- CHILE
        -- SANTIAGO
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '88aaaf99-a444-4e0f-99f6-267b635faa53'  -- SAO - Vargas
                )
GROUP BY 1
            )
            
            



SELECT 

'LATAM' as region,
dc.country,
(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82'  -- BOG - Cedritos
      )
   THEN
      'Bogota'
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4'  -- GDL - Ciudad del Sol
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b'  -- RIO - Camerino
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '88aaaf99-a444-4e0f-99f6-267b635faa53'  -- SAO - Vargas
      )
   then
      'Sao Paulo'
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,

organization_name,
facility_country_code,
organization_id

--ofo_slug,
--date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) < date_add('week', 4, date_trunc('week', DATE(from_unixtime(first_order, facility_timezone)))) as is_new_store,
--is_cancelled,
--subtotal,
--total,




FROM hudi_ingest.analytics_views.customer_orders co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
LEFT JOIN first_order fo 
   ON co.store_id=fo.store_id

 --4 meses   
WHERE date_trunc('day',date(day_partition)) >= DATE_ADD('Day' ,-31, current_date)
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-01'
AND NOT is_cancelled
AND tenant_type = 'VIRTUAL'
-- AND organization_id = '2a694864-3f0c-45e6-8a8d-d6088e0d122f'

                
                GROUP by 1,2,3,4,5,6

"""

print("querying ...")
#por_cliente = ds_trino.fetch_data(conn=conn, sql_str=query_por_cliente, use_cache=False)
print("pulled")

querying ...
pulled


In [208]:
#por_cliente

In [209]:
## GSheets Oauth
print('Init Auth')
#gc = pygsheets.authorize(custom_credentials=my_credentials)


# open the spreedsheet
print('Opening Spreadsheet')

#ss = gc.open_by_key('1Ffh0lxSI31wEckOrbpB1gxgd4ry3GdXO62XvksepGds')

# select sheet

#ws = ss.worksheet('title','clientes_activos')

print('Clearing Range ...')
#ws.clear(start='A', end='x')
print('Range CLEARED')

print('Pasting Info ...')
#ws.set_dataframe(por_cliente, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
